# MAS 649 Midterm
# Group 5
### Austin Gravely, Elias Eskind, Arlet Rodriguez, Cesar Diez, Juliano Torii
### 4/6/21

# Step 1: Loading & Splitting Data

## Loading in Data

In [0]:
btc = spark.read.csv('/FileStore/tables/BTC_USD.csv',inferSchema=True,header=True)
eth = spark.read.csv('/FileStore/tables/ETH_USD.csv',inferSchema=True,header=True)
theta = spark.read.csv('/FileStore/tables/THETA_USD.csv',inferSchema=True,header=True)

## Viewing Data

In [0]:
# Display
display(btc)
display(eth)
display(theta)

Date,Open,High,Low,Close,Volume,ReturnY,OCDiff,Lag1,Lag2,Lag3,Lag4
12/10/19,7397.134277,7424.022949,7246.043945,7278.119629,18249031195,-0.00833902,null,null,null,null,null
12/11/19,7277.197754,7324.15625,7195.527344,7217.427246,16350490689,0.0035618,-1.2668E-4,-0.00833902,null,null,null
12/12/19,7216.73877,7266.639648,7164.741211,7243.134277,18927080224,0.003665581,-9.54E-5,0.0035618,-0.00833902,null,null
12/13/19,7244.662109,7293.560547,7227.122559,7269.68457,17125736940,-0.019947322,2.10891E-4,0.003665581,0.0035618,-0.00833902,null
12/14/19,7268.902832,7308.836426,7097.208984,7124.673828,17137029730,0.003877782,-1.07546E-4,-0.019947322,0.003665581,0.0035618,-0.00833902
12/15/19,7124.239746,7181.075684,6924.375977,7152.301758,16881129804,-0.030734342,-6.09E-5,0.003877782,-0.019947322,0.003665581,0.0035618
12/16/19,7153.663086,7171.168945,6903.682617,6932.480469,20213265950,-0.042115565,1.90298E-4,-0.030734342,0.003877782,-0.019947322,0.003665581
12/17/19,6931.31543,6964.075195,6587.974121,6640.515137,22363804217,0.095819012,-1.68083E-4,-0.042115565,-0.030734342,0.003877782,-0.019947322
12/18/19,6647.698242,7324.984863,6540.049316,7276.802734,31836522778,-0.010163598,0.00108054,0.095819012,-0.042115565,-0.030734342,0.003877782
12/19/19,7277.59082,7346.602539,7041.381836,7202.844238,25904604416,0.002217481,1.08289E-4,-0.010163598,0.095819012,-0.042115565,-0.030734342


Date,Open,High,Low,Close,Volume,ReturnY,OCDiff,Lag1,Lag2,Lag3,Lag4
12/10/19,148.179855,148.564468,144.907959,146.267044,6859512025,-0.018179365,null,null,null,null,null
12/11/19,146.320648,147.139206,143.045364,143.608002,7037180049,0.013898961,3.66346E-4,-0.018179365,null,null,null
12/12/19,143.615662,145.751648,141.436981,145.604004,7890383413,-0.004527733,5.33E-5,0.013898961,-0.018179365,null,null
12/13/19,145.655685,145.857101,143.746521,144.944748,7264810247,-0.01431936,3.54816E-4,-0.004527733,0.013898961,-0.018179365,null
12/14/19,144.953415,145.529083,142.434555,142.869232,7048066973,0.00172016,5.98E-5,-0.01431936,-0.004527733,0.013898961,-0.018179365
12/15/19,142.86499,143.925354,139.426956,143.11499,7235153411,-0.066386903,-2.97E-5,0.00172016,-0.01431936,-0.004527733,0.013898961
12/16/19,143.139526,143.224854,132.456665,133.614029,8992282119,-0.082402575,1.71413E-4,-0.066386903,0.00172016,-0.01431936,-0.004527733
12/17/19,133.647186,134.011536,121.395081,122.603889,9057166141,0.085546267,2.48094E-4,-0.082402575,-0.066386903,0.00172016,-0.01431936
12/18/19,122.656837,133.394165,119.78006,133.092194,11864518321,-0.028334179,4.31676E-4,0.085546267,-0.082402575,-0.066386903,0.00172016
12/19/19,133.05278,134.190643,125.971664,129.321136,9564699140,-0.001972462,-2.96228E-4,-0.028334179,0.085546267,-0.082402575,-0.066386903


Date,Open,High,Low,Close,Volume,ReturnY,OCDiff,Lag1,Lag2,Lag3,Lag4
12/10/19,0.085608,0.085714,0.073951,0.076597,2773792,0.084311396,null,null,null,null,null
12/11/19,0.076638,0.086088,0.076493,0.083055,2147619,0.032785504,5.34983E-4,0.084311396,null,null,null
12/12/19,0.083027,0.08921,0.078864,0.085778,3673322,0.05091049,-3.3724E-4,0.032785504,0.084311396,null,null
12/13/19,0.085842,0.092184,0.084736,0.090145,3681541,-0.016196128,7.45556E-4,0.05091049,0.032785504,0.084311396,null
12/14/19,0.090143,0.090722,0.083036,0.088685,1768192,0.077093082,-2.22E-5,-0.016196128,0.05091049,0.032785504,0.084311396
12/15/19,0.088692,0.098924,0.084474,0.095522,5096621,-0.041770482,7.89E-5,0.077093082,-0.016196128,0.05091049,0.032785504
12/16/19,0.095568,0.100991,0.088694,0.091532,5611689,0.032360267,4.81333E-4,-0.041770482,0.077093082,-0.016196128,0.05091049
12/17/19,0.091532,0.10091,0.089191,0.094494,7840056,0.071464855,0.0,0.032360267,-0.041770482,0.077093082,-0.016196128
12/18/19,0.094494,0.104688,0.091284,0.101247,6151333,0.063873497,0.0,0.071464855,0.032360267,-0.041770482,0.077093082
12/19/19,0.101247,0.112162,0.100012,0.107714,5951808,-0.01707299,0.0,0.063873497,0.071464855,0.032360267,-0.041770482


In [0]:
# View Schema
btc.printSchema()
eth.printSchema()
theta.printSchema()

root
-- Date: string (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: long (nullable = true)
-- ReturnY: double (nullable = true)
-- OCDiff: double (nullable = true)
-- Lag1: double (nullable = true)
-- Lag2: double (nullable = true)
-- Lag3: double (nullable = true)
-- Lag4: double (nullable = true)

root
-- Date: string (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: long (nullable = true)
-- ReturnY: double (nullable = true)
-- OCDiff: double (nullable = true)
-- Lag1: double (nullable = true)
-- Lag2: double (nullable = true)
-- Lag3: double (nullable = true)
-- Lag4: double (nullable = true)

root
-- Date: string (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: integer (nullable = true)
-- ReturnY: double (nullable = true)
-- OCDiff: double (nullable = true)
-- Lag1: double (nullable = true)
-- Lag2: double (nullable = true)
-- Lag3: double (nullable = true)
-- Lag4: double (nullable = true)

## Dropping N/A's

In [0]:
btc2=btc.na.drop()
eth2=eth.na.drop()
theta2=theta.na.drop()

## Changing 'Date' Column from String to Date Datatype

In [0]:
from pyspark.sql.functions import to_date

# Bitcoin
btc2 = btc2.withColumn("Date", to_date(btc2['Date'], "M/d/yy"))
btc2.printSchema()

# Ethereum
eth2 = eth2.withColumn("Date", to_date(eth2['Date'], "M/d/yy"))
eth2.printSchema()

# Theta
theta2 = theta2.withColumn("Date", to_date(theta2['Date'], "M/d/yy"))
theta2.printSchema()

root
-- Date: date (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: long (nullable = true)
-- ReturnY: double (nullable = true)
-- OCDiff: double (nullable = true)
-- Lag1: double (nullable = true)
-- Lag2: double (nullable = true)
-- Lag3: double (nullable = true)
-- Lag4: double (nullable = true)

root
-- Date: date (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: long (nullable = true)
-- ReturnY: double (nullable = true)
-- OCDiff: double (nullable = true)
-- Lag1: double (nullable = true)
-- Lag2: double (nullable = true)
-- Lag3: double (nullable = true)
-- Lag4: double (nullable = true)

root
-- Date: date (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: integer (nullable = true)
-- ReturnY: double (nullable = true)
-- OCDiff: double (nullable = true)
-- Lag1: double (nullable = true)
-- Lag2: double (nullable = true)
-- Lag3: double (nullable = true)
-- Lag4: double (nullable = true)

## Splitting data into Train & Test

### Bitcoin

In [0]:
# Train
train_databtc2 = btc2.limit(316)
display(train_databtc2)

#Test
test_databtc2 = btc2.orderBy(btc2["Date"].desc()).limit(30).orderBy("Date")
display(test_databtc2)

Date,Open,High,Low,Close,Volume,ReturnY,OCDiff,Lag1,Lag2,Lag3,Lag4
2019-12-14,7268.902832,7308.836426,7097.208984,7124.673828,17137029730,0.003877782,-1.07546E-4,-0.019947322,0.003665581,0.0035618,-0.00833902
2019-12-15,7124.239746,7181.075684,6924.375977,7152.301758,16881129804,-0.030734342,-6.09E-5,0.003877782,-0.019947322,0.003665581,0.0035618
2019-12-16,7153.663086,7171.168945,6903.682617,6932.480469,20213265950,-0.042115565,1.90298E-4,-0.030734342,0.003877782,-0.019947322,0.003665581
2019-12-17,6931.31543,6964.075195,6587.974121,6640.515137,22363804217,0.095819012,-1.68083E-4,-0.042115565,-0.030734342,0.003877782,-0.019947322
2019-12-18,6647.698242,7324.984863,6540.049316,7276.802734,31836522778,-0.010163598,0.00108054,0.095819012,-0.042115565,-0.030734342,0.003877782
2019-12-19,7277.59082,7346.602539,7041.381836,7202.844238,25904604416,0.002217481,1.08289E-4,-0.010163598,0.095819012,-0.042115565,-0.030734342
2019-12-20,7208.636719,7257.921875,7086.124023,7218.816406,22633815180,-0.003831337,8.03547E-4,0.002217481,-0.010163598,0.095819012,-0.042115565
2019-12-21,7220.59375,7223.226074,7112.73584,7191.158691,19312552168,0.044558908,2.46149E-4,-0.003831337,0.002217481,-0.010163598,0.095819012
2019-12-22,7191.188477,7518.033203,7167.179199,7511.588867,23134537956,-0.020762644,4.14E-6,0.044558908,-0.003831337,0.002217481,-0.010163598
2019-12-23,7508.902344,7656.17627,7326.192383,7355.628418,27831788041,-0.004499438,-3.57778E-4,-0.020762644,0.044558908,-0.003831337,0.002217481


Date,Open,High,Low,Close,Volume,ReturnY,OCDiff,Lag1,Lag2,Lag3,Lag4
2020-11-10,15332.35059,15450.3291,15124.95996,15290.90234,25574938143,0.026841941,2.29E-6,-0.002701033,-0.009512666,0.043536753,-0.047034084
2020-11-11,15290.90918,15916.26074,15290.00684,15701.33984,29772374934,0.036621327,4.47E-7,0.026841941,-0.002701033,-0.009512666,0.043536753
2020-11-12,15701.29883,16305.00391,15534.77148,16276.34375,34175758344,0.002547553,-2.61E-6,0.036621327,0.026841941,-0.002701033,-0.009512666
2020-11-13,16276.44043,16463.17773,15992.15234,16317.80859,31599492172,-0.015300457,5.94E-6,0.002547553,0.036621327,0.026841941,-0.002701033
2020-11-14,16317.80859,16317.80859,15749.19336,16068.13867,27481710135,-0.007004594,0.0,-0.015300457,0.002547553,0.036621327,0.026841941
2020-11-15,16068.13965,16123.11035,15793.53418,15955.58789,23653867583,0.047665021,6.07E-8,-0.007004594,-0.015300457,0.002547553,0.036621327
2020-11-16,15955.57715,16816.18164,15880.70606,16716.11133,31526766675,0.055592769,-6.73E-7,0.047665021,-0.007004594,-0.015300457,0.002547553
2020-11-17,16685.69141,17782.91992,16564.54492,17645.40625,39006849170,0.008988153,-0.001823114,0.055592769,0.047665021,-0.007004594,-0.015300457
2020-11-18,17645.19141,18393.94922,17352.90625,17804.00586,49064800278,7.3489E-4,-1.22E-5,0.008988153,0.055592769,0.047665021,-0.007004594
2020-11-19,17803.86133,18119.54688,17382.55469,17817.08984,36985055355,0.045137821,-8.12E-6,7.3489E-4,0.008988153,0.055592769,0.047665021


### Ethereum

In [0]:
# Train
train_dataeth2 = eth2.limit(316)
display(train_dataeth2)

#Test
test_dataeth2 = eth2.orderBy(eth2["Date"].desc()).limit(30).orderBy("Date")
display(test_dataeth2)

Date,Open,High,Low,Close,Volume,ReturnY,OCDiff,Lag1,Lag2,Lag3,Lag4
2019-12-14,144.953415,145.529083,142.434555,142.869232,7048066973,0.00172016,5.98E-5,-0.01431936,-0.004527733,0.013898961,-0.018179365
2019-12-15,142.86499,143.925354,139.426956,143.11499,7235153411,-0.066386903,-2.97E-5,0.00172016,-0.01431936,-0.004527733,0.013898961
2019-12-16,143.139526,143.224854,132.456665,133.614029,8992282119,-0.082402575,1.71413E-4,-0.066386903,0.00172016,-0.01431936,-0.004527733
2019-12-17,133.647186,134.011536,121.395081,122.603889,9057166141,0.085546267,2.48094E-4,-0.082402575,-0.066386903,0.00172016,-0.01431936
2019-12-18,122.656837,133.394165,119.78006,133.092194,11864518321,-0.028334179,4.31676E-4,0.085546267,-0.082402575,-0.066386903,0.00172016
2019-12-19,133.05278,134.190643,125.971664,129.321136,9564699140,-0.001972462,-2.96228E-4,-0.028334179,0.085546267,-0.082402575,-0.066386903
2019-12-20,129.384369,129.756302,126.985352,129.066055,8173594150,-0.007245158,4.88722E-4,-0.001972462,-0.028334179,0.085546267,-0.082402575
2019-12-21,129.095963,129.147659,127.612091,128.130951,7449064703,0.033103337,2.31673E-4,-0.007245158,-0.001972462,-0.028334179,0.085546267
2019-12-22,128.098129,133.160034,127.92849,132.372513,9121058481,-0.028392775,-2.56225E-4,0.033103337,-0.007245158,-0.001972462,-0.028334179
2019-12-23,132.306992,134.73175,127.416718,128.61409,9622497693,0.003497276,-4.95219E-4,-0.028392775,0.033103337,-0.007245158,-0.001972462


Date,Open,High,Low,Close,Volume,ReturnY,OCDiff,Lag1,Lag2,Lag3,Lag4
2020-11-10,444.166382,453.758362,439.600128,449.679626,12090381666,0.029534171,7.49E-6,0.012420148,-0.020706923,0.040948144,-0.041797575
2020-11-11,449.679657,473.578857,449.524933,462.960541,14075403511,-0.004223386,6.89E-8,0.029534171,0.012420148,-0.020706923,0.040948144
2020-11-12,462.959534,467.677826,452.072418,461.00528,12877327234,0.029546633,-2.18E-6,-0.004223386,0.029534171,0.012420148,-0.020706923
2020-11-13,461.005493,475.217255,457.298248,474.626434,13191505725,-0.030501026,4.62E-7,0.029546633,-0.004223386,0.029534171,0.012420148
2020-11-14,474.626434,475.161438,452.986084,460.149841,10312037942,-0.027362302,0.0,-0.030501026,0.029546633,-0.004223386,0.029534171
2020-11-15,460.149902,460.99408,440.254333,447.559082,10308617165,0.027663892,1.33E-7,-0.027362302,-0.030501026,0.029546633,-0.004223386
2020-11-16,447.55899,463.831024,445.501617,459.940308,11441239444,0.044396563,-2.06E-7,0.027663892,-0.027362302,-0.030501026,0.029546633
2020-11-17,464.406647,482.232208,460.776611,480.360077,14593057877,-0.001823647,0.009617302,0.044396563,0.027663892,-0.027362302,-0.030501026
2020-11-18,480.346832,491.999908,465.830963,479.48407,17880199224,-0.016379351,-2.76E-5,-0.001823647,0.044396563,0.027663892,-0.027362302
2020-11-19,479.481018,480.121735,465.704254,471.630432,12473929218,0.080813564,-6.37E-6,-0.016379351,-0.001823647,0.044396563,0.027663892


### Theta

In [0]:
# Train
train_datatheta2 = theta2.limit(316)
display(train_datatheta2)

#Test
test_datatheta2 = theta2.orderBy(theta2["Date"].desc()).limit(30).orderBy("Date")
display(test_datatheta2)

Date,Open,High,Low,Close,Volume,ReturnY,OCDiff,Lag1,Lag2,Lag3,Lag4
2019-12-14,0.090143,0.090722,0.083036,0.088685,1768192,0.077093082,-2.22E-5,-0.016196128,0.05091049,0.032785504,0.084311396
2019-12-15,0.088692,0.098924,0.084474,0.095522,5096621,-0.041770482,7.89E-5,0.077093082,-0.016196128,0.05091049,0.032785504
2019-12-16,0.095568,0.100991,0.088694,0.091532,5611689,0.032360267,4.81333E-4,-0.041770482,0.077093082,-0.016196128,0.05091049
2019-12-17,0.091532,0.10091,0.089191,0.094494,7840056,0.071464855,0.0,0.032360267,-0.041770482,0.077093082,-0.016196128
2019-12-18,0.094494,0.104688,0.091284,0.101247,6151333,0.063873497,0.0,0.071464855,0.032360267,-0.041770482,0.077093082
2019-12-19,0.101247,0.112162,0.100012,0.107714,5951808,-0.01707299,0.0,0.063873497,0.071464855,0.032360267,-0.041770482
2019-12-20,0.107714,0.109948,0.103282,0.105875,2764379,-0.05790791,0.0,-0.01707299,0.063873497,0.071464855,0.032360267
2019-12-21,0.105819,0.10645,0.098511,0.099744,2763499,-0.024091675,-5.29206E-4,-0.05790791,-0.01707299,0.063873497,0.071464855
2019-12-22,0.099887,0.106496,0.095886,0.097341,3098911,-0.070463628,0.001431618,-0.024091675,-0.05790791,-0.01707299,0.063873497
2019-12-23,0.097331,0.100891,0.090466,0.090482,2753940,0.029972812,-1.02742E-4,-0.070463628,-0.024091675,-0.05790791,-0.01707299


Date,Open,High,Low,Close,Volume,ReturnY,OCDiff,Lag1,Lag2,Lag3,Lag4
2020-11-10,0.641028,0.657987,0.623313,0.641003,19591349,-0.044586375,-1.56E-6,-4.06E-5,-0.035165933,0.038349314,-0.053501138
2020-11-11,0.641003,0.653566,0.612423,0.612423,18730118,-0.017145013,0.0,-0.044586375,-4.06E-5,-0.035165933,0.038349314
2020-11-12,0.612423,0.626791,0.590405,0.601923,16643669,0.052284096,0.0,-0.017145013,-0.044586375,-4.06E-5,-0.035165933
2020-11-13,0.601928,0.634515,0.595535,0.633394,22198608,-0.033803604,8.31E-6,0.052284096,-0.017145013,-0.044586375,-4.06E-5
2020-11-14,0.633394,0.63766,0.595949,0.611983,9700736,-0.013719335,0.0,-0.033803604,0.052284096,-0.017145013,-0.044586375
2020-11-15,0.611983,0.62249,0.59375,0.603587,8911596,0.025005509,0.0,-0.013719335,-0.033803604,0.052284096,-0.017145013
2020-11-16,0.603587,0.629743,0.596472,0.61868,10990771,0.056659339,0.0,0.025005509,-0.013719335,-0.033803604,0.052284096
2020-11-17,0.629428,0.654897,0.621839,0.653734,17996693,-0.044674134,0.017075821,0.056659339,0.025005509,-0.013719335,-0.033803604
2020-11-18,0.653732,0.677797,0.610337,0.624529,15617187,-0.001943865,-3.06E-6,-0.044674134,0.056659339,0.025005509,-0.013719335
2020-11-19,0.624534,0.639607,0.61423,0.623315,10601771,0.027804561,8.01E-6,-0.001943865,-0.044674134,0.056659339,0.025005509


## Transforming Data

In [0]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ['Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'OCDiff',
 'Lag1',
 'Lag2',
 'Lag3',
 'Lag4'], outputCol = "features")

In [0]:
# Bitcoin
outputTrainbtc2 = assembler.transform(train_databtc2)
display(outputTrainbtc2)

outputTestbtc2 = assembler.transform(test_databtc2)
display(outputTestbtc2)

# Ethereum
outputTraineth2 = assembler.transform(train_dataeth2)
display(outputTraineth2)

outputTesteth2  = assembler.transform(test_dataeth2)
display(outputTesteth2)

# Theta
outputTraintheta2 = assembler.transform(train_datatheta2)
display(outputTraintheta2)

outputTesttheta2  = assembler.transform(test_datatheta2)
display(outputTesttheta2)

Date,Open,High,Low,Close,Volume,ReturnY,OCDiff,Lag1,Lag2,Lag3,Lag4,features
2019-12-14,7268.902832,7308.836426,7097.208984,7124.673828,17137029730,0.003877782,-1.07546E-4,-0.019947322,0.003665581,0.0035618,-0.00833902,"Map(vectorType -> dense, length -> 10, values -> List(7268.902832, 7308.836426, 7097.208984, 7124.673828, 1.713702973E10, -1.07546E-4, -0.019947322, 0.003665581, 0.0035618, -0.00833902))"
2019-12-15,7124.239746,7181.075684,6924.375977,7152.301758,16881129804,-0.030734342,-6.09E-5,0.003877782,-0.019947322,0.003665581,0.0035618,"Map(vectorType -> dense, length -> 10, values -> List(7124.239746, 7181.075684, 6924.375977, 7152.301758, 1.6881129804E10, -6.09E-5, 0.003877782, -0.019947322, 0.003665581, 0.0035618))"
2019-12-16,7153.663086,7171.168945,6903.682617,6932.480469,20213265950,-0.042115565,1.90298E-4,-0.030734342,0.003877782,-0.019947322,0.003665581,"Map(vectorType -> dense, length -> 10, values -> List(7153.663086, 7171.168945, 6903.682617, 6932.480469, 2.021326595E10, 1.90298E-4, -0.030734342, 0.003877782, -0.019947322, 0.003665581))"
2019-12-17,6931.31543,6964.075195,6587.974121,6640.515137,22363804217,0.095819012,-1.68083E-4,-0.042115565,-0.030734342,0.003877782,-0.019947322,"Map(vectorType -> dense, length -> 10, values -> List(6931.31543, 6964.075195, 6587.974121, 6640.515137, 2.2363804217E10, -1.68083E-4, -0.042115565, -0.030734342, 0.003877782, -0.019947322))"
2019-12-18,6647.698242,7324.984863,6540.049316,7276.802734,31836522778,-0.010163598,0.00108054,0.095819012,-0.042115565,-0.030734342,0.003877782,"Map(vectorType -> dense, length -> 10, values -> List(6647.698242, 7324.984863, 6540.049316, 7276.802734, 3.1836522778E10, 0.00108054, 0.095819012, -0.042115565, -0.030734342, 0.003877782))"
2019-12-19,7277.59082,7346.602539,7041.381836,7202.844238,25904604416,0.002217481,1.08289E-4,-0.010163598,0.095819012,-0.042115565,-0.030734342,"Map(vectorType -> dense, length -> 10, values -> List(7277.59082, 7346.602539, 7041.381836, 7202.844238, 2.5904604416E10, 1.08289E-4, -0.010163598, 0.095819012, -0.042115565, -0.030734342))"
2019-12-20,7208.636719,7257.921875,7086.124023,7218.816406,22633815180,-0.003831337,8.03547E-4,0.002217481,-0.010163598,0.095819012,-0.042115565,"Map(vectorType -> dense, length -> 10, values -> List(7208.636719, 7257.921875, 7086.124023, 7218.816406, 2.263381518E10, 8.03547E-4, 0.002217481, -0.010163598, 0.095819012, -0.042115565))"
2019-12-21,7220.59375,7223.226074,7112.73584,7191.158691,19312552168,0.044558908,2.46149E-4,-0.003831337,0.002217481,-0.010163598,0.095819012,"Map(vectorType -> dense, length -> 10, values -> List(7220.59375, 7223.226074, 7112.73584, 7191.158691, 1.9312552168E10, 2.46149E-4, -0.003831337, 0.002217481, -0.010163598, 0.095819012))"
2019-12-22,7191.188477,7518.033203,7167.179199,7511.588867,23134537956,-0.020762644,4.14E-6,0.044558908,-0.003831337,0.002217481,-0.010163598,"Map(vectorType -> dense, length -> 10, values -> List(7191.188477, 7518.033203, 7167.179199, 7511.588867, 2.3134537956E10, 4.14E-6, 0.044558908, -0.003831337, 0.002217481, -0.010163598))"
2019-12-23,7508.902344,7656.17627,7326.192383,7355.628418,27831788041,-0.004499438,-3.57778E-4,-0.020762644,0.044558908,-0.003831337,0.002217481,"Map(vectorType -> dense, length -> 10, values -> List(7508.902344, 7656.17627, 7326.192383, 7355.628418, 2.7831788041E10, -3.57778E-4, -0.020762644, 0.044558908, -0.003831337, 0.002217481))"


Date,Open,High,Low,Close,Volume,ReturnY,OCDiff,Lag1,Lag2,Lag3,Lag4,features
2020-11-10,15332.35059,15450.3291,15124.95996,15290.90234,25574938143,0.026841941,2.29E-6,-0.002701033,-0.009512666,0.043536753,-0.047034084,"Map(vectorType -> dense, length -> 10, values -> List(15332.35059, 15450.3291, 15124.95996, 15290.90234, 2.5574938143E10, 2.29E-6, -0.002701033, -0.009512666, 0.043536753, -0.047034084))"
2020-11-11,15290.90918,15916.26074,15290.00684,15701.33984,29772374934,0.036621327,4.47E-7,0.026841941,-0.002701033,-0.009512666,0.043536753,"Map(vectorType -> dense, length -> 10, values -> List(15290.90918, 15916.26074, 15290.00684, 15701.33984, 2.9772374934E10, 4.47E-7, 0.026841941, -0.002701033, -0.009512666, 0.043536753))"
2020-11-12,15701.29883,16305.00391,15534.77148,16276.34375,34175758344,0.002547553,-2.61E-6,0.036621327,0.026841941,-0.002701033,-0.009512666,"Map(vectorType -> dense, length -> 10, values -> List(15701.29883, 16305.00391, 15534.77148, 16276.34375, 3.4175758344E10, -2.61E-6, 0.036621327, 0.026841941, -0.002701033, -0.009512666))"
2020-11-13,16276.44043,16463.17773,15992.15234,16317.80859,31599492172,-0.015300457,5.94E-6,0.002547553,0.036621327,0.026841941,-0.002701033,"Map(vectorType -> dense, length -> 10, values -> List(16276.44043, 16463.17773, 15992.15234, 16317.80859, 3.1599492172E10, 5.94E-6, 0.002547553, 0.036621327, 0.026841941, -0.002701033))"
2020-11-14,16317.80859,16317.80859,15749.19336,16068.13867,27481710135,-0.007004594,0.0,-0.015300457,0.002547553,0.036621327,0.026841941,"Map(vectorType -> dense, length -> 10, values -> List(16317.80859, 16317.80859, 15749.19336, 16068.13867, 2.7481710135E10, 0.0, -0.015300457, 0.002547553, 0.036621327, 0.026841941))"
2020-11-15,16068.13965,16123.11035,15793.53418,15955.58789,23653867583,0.047665021,6.07E-8,-0.007004594,-0.015300457,0.002547553,0.036621327,"Map(vectorType -> dense, length -> 10, values -> List(16068.13965, 16123.11035, 15793.53418, 15955.58789, 2.3653867583E10, 6.07E-8, -0.007004594, -0.015300457, 0.002547553, 0.036621327))"
2020-11-16,15955.57715,16816.18164,15880.70606,16716.11133,31526766675,0.055592769,-6.73E-7,0.047665021,-0.007004594,-0.015300457,0.002547553,"Map(vectorType -> dense, length -> 10, values -> List(15955.57715, 16816.18164, 15880.70606, 16716.11133, 3.1526766675E10, -6.73E-7, 0.047665021, -0.007004594, -0.015300457, 0.002547553))"
2020-11-17,16685.69141,17782.91992,16564.54492,17645.40625,39006849170,0.008988153,-0.001823114,0.055592769,0.047665021,-0.007004594,-0.015300457,"Map(vectorType -> dense, length -> 10, values -> List(16685.69141, 17782.91992, 16564.54492, 17645.40625, 3.900684917E10, -0.001823114, 0.055592769, 0.047665021, -0.007004594, -0.015300457))"
2020-11-18,17645.19141,18393.94922,17352.90625,17804.00586,49064800278,7.3489E-4,-1.22E-5,0.008988153,0.055592769,0.047665021,-0.007004594,"Map(vectorType -> dense, length -> 10, values -> List(17645.19141, 18393.94922, 17352.90625, 17804.00586, 4.9064800278E10, -1.22E-5, 0.008988153, 0.055592769, 0.047665021, -0.007004594))"
2020-11-19,17803.86133,18119.54688,17382.55469,17817.08984,36985055355,0.045137821,-8.12E-6,7.3489E-4,0.008988153,0.055592769,0.047665021,"Map(vectorType -> dense, length -> 10, values -> List(17803.86133, 18119.54688, 17382.55469, 17817.08984, 3.6985055355E10, -8.12E-6, 7.3489E-4, 0.008988153, 0.055592769, 0.047665021))"


Date,Open,High,Low,Close,Volume,ReturnY,OCDiff,Lag1,Lag2,Lag3,Lag4,features
2019-12-14,144.953415,145.529083,142.434555,142.869232,7048066973,0.00172016,5.98E-5,-0.01431936,-0.004527733,0.013898961,-0.018179365,"Map(vectorType -> dense, length -> 10, values -> List(144.953415, 145.529083, 142.434555, 142.869232, 7.048066973E9, 5.98E-5, -0.01431936, -0.004527733, 0.013898961, -0.018179365))"
2019-12-15,142.86499,143.925354,139.426956,143.11499,7235153411,-0.066386903,-2.97E-5,0.00172016,-0.01431936,-0.004527733,0.013898961,"Map(vectorType -> dense, length -> 10, values -> List(142.86499, 143.925354, 139.426956, 143.11499, 7.235153411E9, -2.97E-5, 0.00172016, -0.01431936, -0.004527733, 0.013898961))"
2019-12-16,143.139526,143.224854,132.456665,133.614029,8992282119,-0.082402575,1.71413E-4,-0.066386903,0.00172016,-0.01431936,-0.004527733,"Map(vectorType -> dense, length -> 10, values -> List(143.139526, 143.224854, 132.456665, 133.614029, 8.992282119E9, 1.71413E-4, -0.066386903, 0.00172016, -0.01431936, -0.004527733))"
2019-12-17,133.647186,134.011536,121.395081,122.603889,9057166141,0.085546267,2.48094E-4,-0.082402575,-0.066386903,0.00172016,-0.01431936,"Map(vectorType -> dense, length -> 10, values -> List(133.647186, 134.011536, 121.395081, 122.603889, 9.057166141E9, 2.48094E-4, -0.082402575, -0.066386903, 0.00172016, -0.01431936))"
2019-12-18,122.656837,133.394165,119.78006,133.092194,11864518321,-0.028334179,4.31676E-4,0.085546267,-0.082402575,-0.066386903,0.00172016,"Map(vectorType -> dense, length -> 10, values -> List(122.656837, 133.394165, 119.78006, 133.092194, 1.1864518321E10, 4.31676E-4, 0.085546267, -0.082402575, -0.066386903, 0.00172016))"
2019-12-19,133.05278,134.190643,125.971664,129.321136,9564699140,-0.001972462,-2.96228E-4,-0.028334179,0.085546267,-0.082402575,-0.066386903,"Map(vectorType -> dense, length -> 10, values -> List(133.05278, 134.190643, 125.971664, 129.321136, 9.56469914E9, -2.96228E-4, -0.028334179, 0.085546267, -0.082402575, -0.066386903))"
2019-12-20,129.384369,129.756302,126.985352,129.066055,8173594150,-0.007245158,4.88722E-4,-0.001972462,-0.028334179,0.085546267,-0.082402575,"Map(vectorType -> dense, length -> 10, values -> List(129.384369, 129.756302, 126.985352, 129.066055, 8.17359415E9, 4.88722E-4, -0.001972462, -0.028334179, 0.085546267, -0.082402575))"
2019-12-21,129.095963,129.147659,127.612091,128.130951,7449064703,0.033103337,2.31673E-4,-0.007245158,-0.001972462,-0.028334179,0.085546267,"Map(vectorType -> dense, length -> 10, values -> List(129.095963, 129.147659, 127.612091, 128.130951, 7.449064703E9, 2.31673E-4, -0.007245158, -0.001972462, -0.028334179, 0.085546267))"
2019-12-22,128.098129,133.160034,127.92849,132.372513,9121058481,-0.028392775,-2.56225E-4,0.033103337,-0.007245158,-0.001972462,-0.028334179,"Map(vectorType -> dense, length -> 10, values -> List(128.098129, 133.160034, 127.92849, 132.372513, 9.121058481E9, -2.56225E-4, 0.033103337, -0.007245158, -0.001972462, -0.028334179))"
2019-12-23,132.306992,134.73175,127.416718,128.61409,9622497693,0.003497276,-4.95219E-4,-0.028392775,0.033103337,-0.007245158,-0.001972462,"Map(vectorType -> dense, length -> 10, values -> List(132.306992, 134.73175, 127.416718, 128.61409, 9.622497693E9, -4.95219E-4, -0.028392775, 0.033103337, -0.007245158, -0.001972462))"


Date,Open,High,Low,Close,Volume,ReturnY,OCDiff,Lag1,Lag2,Lag3,Lag4,features
2020-11-10,444.166382,453.758362,439.600128,449.679626,12090381666,0.029534171,7.49E-6,0.012420148,-0.020706923,0.040948144,-0.041797575,"Map(vectorType -> dense, length -> 10, values -> List(444.166382, 453.758362, 439.600128, 449.679626, 1.2090381666E10, 7.49E-6, 0.012420148, -0.020706923, 0.040948144, -0.041797575))"
2020-11-11,449.679657,473.578857,449.524933,462.960541,14075403511,-0.004223386,6.89E-8,0.029534171,0.012420148,-0.020706923,0.040948144,"Map(vectorType -> dense, length -> 10, values -> List(449.679657, 473.578857, 449.524933, 462.960541, 1.4075403511E10, 6.89E-8, 0.029534171, 0.012420148, -0.020706923, 0.040948144))"
2020-11-12,462.959534,467.677826,452.072418,461.00528,12877327234,0.029546633,-2.18E-6,-0.004223386,0.029534171,0.012420148,-0.020706923,"Map(vectorType -> dense, length -> 10, values -> List(462.959534, 467.677826, 452.072418, 461.00528, 1.2877327234E10, -2.18E-6, -0.004223386, 0.029534171, 0.012420148, -0.020706923))"
2020-11-13,461.005493,475.217255,457.298248,474.626434,13191505725,-0.030501026,4.62E-7,0.029546633,-0.004223386,0.029534171,0.012420148,"Map(vectorType -> dense, length -> 10, values -> List(461.005493, 475.217255, 457.298248, 474.626434, 1.3191505725E10, 4.62E-7, 0.029546633, -0.004223386, 0.029534171, 0.012420148))"
2020-11-14,474.626434,475.161438,452.986084,460.149841,10312037942,-0.027362302,0.0,-0.030501026,0.029546633,-0.004223386,0.029534171,"Map(vectorType -> dense, length -> 10, values -> List(474.626434, 475.161438, 452.986084, 460.149841, 1.0312037942E10, 0.0, -0.030501026, 0.029546633, -0.004223386, 0.029534171))"
2020-11-15,460.149902,460.99408,440.254333,447.559082,10308617165,0.027663892,1.33E-7,-0.027362302,-0.030501026,0.029546633,-0.004223386,"Map(vectorType -> dense, length -> 10, values -> List(460.149902, 460.99408, 440.254333, 447.559082, 1.0308617165E10, 1.33E-7, -0.027362302, -0.030501026, 0.029546633, -0.004223386))"
2020-11-16,447.55899,463.831024,445.501617,459.940308,11441239444,0.044396563,-2.06E-7,0.027663892,-0.027362302,-0.030501026,0.029546633,"Map(vectorType -> dense, length -> 10, values -> List(447.55899, 463.831024, 445.501617, 459.940308, 1.1441239444E10, -2.06E-7, 0.027663892, -0.027362302, -0.030501026, 0.029546633))"
2020-11-17,464.406647,482.232208,460.776611,480.360077,14593057877,-0.001823647,0.009617302,0.044396563,0.027663892,-0.027362302,-0.030501026,"Map(vectorType -> dense, length -> 10, values -> List(464.406647, 482.232208, 460.776611, 480.360077, 1.4593057877E10, 0.009617302, 0.044396563, 0.027663892, -0.027362302, -0.030501026))"
2020-11-18,480.346832,491.999908,465.830963,479.48407,17880199224,-0.016379351,-2.76E-5,-0.001823647,0.044396563,0.027663892,-0.027362302,"Map(vectorType -> dense, length -> 10, values -> List(480.346832, 491.999908, 465.830963, 479.48407, 1.7880199224E10, -2.76E-5, -0.001823647, 0.044396563, 0.027663892, -0.027362302))"
2020-11-19,479.481018,480.121735,465.704254,471.630432,12473929218,0.080813564,-6.37E-6,-0.016379351,-0.001823647,0.044396563,0.027663892,"Map(vectorType -> dense, length -> 10, values -> List(479.481018, 480.121735, 465.704254, 471.630432, 1.2473929218E10, -6.37E-6, -0.016379351, -0.001823647, 0.044396563, 0.027663892))"


Date,Open,High,Low,Close,Volume,ReturnY,OCDiff,Lag1,Lag2,Lag3,Lag4,features
2019-12-14,0.090143,0.090722,0.083036,0.088685,1768192,0.077093082,-2.22E-5,-0.016196128,0.05091049,0.032785504,0.084311396,"Map(vectorType -> dense, length -> 10, values -> List(0.090143, 0.090722, 0.083036, 0.088685, 1768192.0, -2.22E-5, -0.016196128, 0.05091049, 0.032785504, 0.084311396))"
2019-12-15,0.088692,0.098924,0.084474,0.095522,5096621,-0.041770482,7.89E-5,0.077093082,-0.016196128,0.05091049,0.032785504,"Map(vectorType -> dense, length -> 10, values -> List(0.088692, 0.098924, 0.084474, 0.095522, 5096621.0, 7.89E-5, 0.077093082, -0.016196128, 0.05091049, 0.032785504))"
2019-12-16,0.095568,0.100991,0.088694,0.091532,5611689,0.032360267,4.81333E-4,-0.041770482,0.077093082,-0.016196128,0.05091049,"Map(vectorType -> dense, length -> 10, values -> List(0.095568, 0.100991, 0.088694, 0.091532, 5611689.0, 4.81333E-4, -0.041770482, 0.077093082, -0.016196128, 0.05091049))"
2019-12-17,0.091532,0.10091,0.089191,0.094494,7840056,0.071464855,0.0,0.032360267,-0.041770482,0.077093082,-0.016196128,"Map(vectorType -> dense, length -> 10, values -> List(0.091532, 0.10091, 0.089191, 0.094494, 7840056.0, 0.0, 0.032360267, -0.041770482, 0.077093082, -0.016196128))"
2019-12-18,0.094494,0.104688,0.091284,0.101247,6151333,0.063873497,0.0,0.071464855,0.032360267,-0.041770482,0.077093082,"Map(vectorType -> dense, length -> 10, values -> List(0.094494, 0.104688, 0.091284, 0.101247, 6151333.0, 0.0, 0.071464855, 0.032360267, -0.041770482, 0.077093082))"
2019-12-19,0.101247,0.112162,0.100012,0.107714,5951808,-0.01707299,0.0,0.063873497,0.071464855,0.032360267,-0.041770482,"Map(vectorType -> dense, length -> 10, values -> List(0.101247, 0.112162, 0.100012, 0.107714, 5951808.0, 0.0, 0.063873497, 0.071464855, 0.032360267, -0.041770482))"
2019-12-20,0.107714,0.109948,0.103282,0.105875,2764379,-0.05790791,0.0,-0.01707299,0.063873497,0.071464855,0.032360267,"Map(vectorType -> dense, length -> 10, values -> List(0.107714, 0.109948, 0.103282, 0.105875, 2764379.0, 0.0, -0.01707299, 0.063873497, 0.071464855, 0.032360267))"
2019-12-21,0.105819,0.10645,0.098511,0.099744,2763499,-0.024091675,-5.29206E-4,-0.05790791,-0.01707299,0.063873497,0.071464855,"Map(vectorType -> dense, length -> 10, values -> List(0.105819, 0.10645, 0.098511, 0.099744, 2763499.0, -5.29206E-4, -0.05790791, -0.01707299, 0.063873497, 0.071464855))"
2019-12-22,0.099887,0.106496,0.095886,0.097341,3098911,-0.070463628,0.001431618,-0.024091675,-0.05790791,-0.01707299,0.063873497,"Map(vectorType -> dense, length -> 10, values -> List(0.099887, 0.106496, 0.095886, 0.097341, 3098911.0, 0.001431618, -0.024091675, -0.05790791, -0.01707299, 0.063873497))"
2019-12-23,0.097331,0.100891,0.090466,0.090482,2753940,0.029972812,-1.02742E-4,-0.070463628,-0.024091675,-0.05790791,-0.01707299,"Map(vectorType -> dense, length -> 10, values -> List(0.097331, 0.100891, 0.090466, 0.090482, 2753940.0, -1.02742E-4, -0.070463628, -0.024091675, -0.05790791, -0.01707299))"


Date,Open,High,Low,Close,Volume,ReturnY,OCDiff,Lag1,Lag2,Lag3,Lag4,features
2020-11-10,0.641028,0.657987,0.623313,0.641003,19591349,-0.044586375,-1.56E-6,-4.06E-5,-0.035165933,0.038349314,-0.053501138,"Map(vectorType -> dense, length -> 10, values -> List(0.641028, 0.657987, 0.623313, 0.641003, 1.9591349E7, -1.56E-6, -4.06E-5, -0.035165933, 0.038349314, -0.053501138))"
2020-11-11,0.641003,0.653566,0.612423,0.612423,18730118,-0.017145013,0.0,-0.044586375,-4.06E-5,-0.035165933,0.038349314,"Map(vectorType -> dense, length -> 10, values -> List(0.641003, 0.653566, 0.612423, 0.612423, 1.8730118E7, 0.0, -0.044586375, -4.06E-5, -0.035165933, 0.038349314))"
2020-11-12,0.612423,0.626791,0.590405,0.601923,16643669,0.052284096,0.0,-0.017145013,-0.044586375,-4.06E-5,-0.035165933,"Map(vectorType -> dense, length -> 10, values -> List(0.612423, 0.626791, 0.590405, 0.601923, 1.6643669E7, 0.0, -0.017145013, -0.044586375, -4.06E-5, -0.035165933))"
2020-11-13,0.601928,0.634515,0.595535,0.633394,22198608,-0.033803604,8.31E-6,0.052284096,-0.017145013,-0.044586375,-4.06E-5,"Map(vectorType -> dense, length -> 10, values -> List(0.601928, 0.634515, 0.595535, 0.633394, 2.2198608E7, 8.31E-6, 0.052284096, -0.017145013, -0.044586375, -4.06E-5))"
2020-11-14,0.633394,0.63766,0.595949,0.611983,9700736,-0.013719335,0.0,-0.033803604,0.052284096,-0.017145013,-0.044586375,"Map(vectorType -> dense, length -> 10, values -> List(0.633394, 0.63766, 0.595949, 0.611983, 9700736.0, 0.0, -0.033803604, 0.052284096, -0.017145013, -0.044586375))"
2020-11-15,0.611983,0.62249,0.59375,0.603587,8911596,0.025005509,0.0,-0.013719335,-0.033803604,0.052284096,-0.017145013,"Map(vectorType -> dense, length -> 10, values -> List(0.611983, 0.62249, 0.59375, 0.603587, 8911596.0, 0.0, -0.013719335, -0.033803604, 0.052284096, -0.017145013))"
2020-11-16,0.603587,0.629743,0.596472,0.61868,10990771,0.056659339,0.0,0.025005509,-0.013719335,-0.033803604,0.052284096,"Map(vectorType -> dense, length -> 10, values -> List(0.603587, 0.629743, 0.596472, 0.61868, 1.0990771E7, 0.0, 0.025005509, -0.013719335, -0.033803604, 0.052284096))"
2020-11-17,0.629428,0.654897,0.621839,0.653734,17996693,-0.044674134,0.017075821,0.056659339,0.025005509,-0.013719335,-0.033803604,"Map(vectorType -> dense, length -> 10, values -> List(0.629428, 0.654897, 0.621839, 0.653734, 1.7996693E7, 0.017075821, 0.056659339, 0.025005509, -0.013719335, -0.033803604))"
2020-11-18,0.653732,0.677797,0.610337,0.624529,15617187,-0.001943865,-3.06E-6,-0.044674134,0.056659339,0.025005509,-0.013719335,"Map(vectorType -> dense, length -> 10, values -> List(0.653732, 0.677797, 0.610337, 0.624529, 1.5617187E7, -3.06E-6, -0.044674134, 0.056659339, 0.025005509, -0.013719335))"
2020-11-19,0.624534,0.639607,0.61423,0.623315,10601771,0.027804561,8.01E-6,-0.001943865,-0.044674134,0.056659339,0.025005509,"Map(vectorType -> dense, length -> 10, values -> List(0.624534, 0.639607, 0.61423, 0.623315, 1.0601771E7, 8.01E-6, -0.001943865, -0.044674134, 0.056659339, 0.025005509))"


## Final Datasets

### Bitcoin

In [0]:
# Final Train
trainBtc = outputTrainbtc2.select("features", "ReturnY", "Date")
display(trainBtc)

# Final Test
testBtc = outputTestbtc2.select("features", "ReturnY", "Date")
display(testBtc)

# Displaying
display(trainBtc.describe())
display(testBtc.describe())

features,ReturnY,Date
"Map(vectorType -> dense, length -> 10, values -> List(7268.902832, 7308.836426, 7097.208984, 7124.673828, 1.713702973E10, -1.07546E-4, -0.019947322, 0.003665581, 0.0035618, -0.00833902))",0.003877782,2019-12-14
"Map(vectorType -> dense, length -> 10, values -> List(7124.239746, 7181.075684, 6924.375977, 7152.301758, 1.6881129804E10, -6.09E-5, 0.003877782, -0.019947322, 0.003665581, 0.0035618))",-0.030734342,2019-12-15
"Map(vectorType -> dense, length -> 10, values -> List(7153.663086, 7171.168945, 6903.682617, 6932.480469, 2.021326595E10, 1.90298E-4, -0.030734342, 0.003877782, -0.019947322, 0.003665581))",-0.042115565,2019-12-16
"Map(vectorType -> dense, length -> 10, values -> List(6931.31543, 6964.075195, 6587.974121, 6640.515137, 2.2363804217E10, -1.68083E-4, -0.042115565, -0.030734342, 0.003877782, -0.019947322))",0.095819012,2019-12-17
"Map(vectorType -> dense, length -> 10, values -> List(6647.698242, 7324.984863, 6540.049316, 7276.802734, 3.1836522778E10, 0.00108054, 0.095819012, -0.042115565, -0.030734342, 0.003877782))",-0.010163598,2019-12-18
"Map(vectorType -> dense, length -> 10, values -> List(7277.59082, 7346.602539, 7041.381836, 7202.844238, 2.5904604416E10, 1.08289E-4, -0.010163598, 0.095819012, -0.042115565, -0.030734342))",0.002217481,2019-12-19
"Map(vectorType -> dense, length -> 10, values -> List(7208.636719, 7257.921875, 7086.124023, 7218.816406, 2.263381518E10, 8.03547E-4, 0.002217481, -0.010163598, 0.095819012, -0.042115565))",-0.003831337,2019-12-20
"Map(vectorType -> dense, length -> 10, values -> List(7220.59375, 7223.226074, 7112.73584, 7191.158691, 1.9312552168E10, 2.46149E-4, -0.003831337, 0.002217481, -0.010163598, 0.095819012))",0.044558908,2019-12-21
"Map(vectorType -> dense, length -> 10, values -> List(7191.188477, 7518.033203, 7167.179199, 7511.588867, 2.3134537956E10, 4.14E-6, 0.044558908, -0.003831337, 0.002217481, -0.010163598))",-0.020762644,2019-12-22
"Map(vectorType -> dense, length -> 10, values -> List(7508.902344, 7656.17627, 7326.192383, 7355.628418, 2.7831788041E10, -3.57778E-4, -0.020762644, 0.044558908, -0.003831337, 0.002217481))",-0.004499438,2019-12-23


features,ReturnY,Date
"Map(vectorType -> dense, length -> 10, values -> List(15332.35059, 15450.3291, 15124.95996, 15290.90234, 2.5574938143E10, 2.29E-6, -0.002701033, -0.009512666, 0.043536753, -0.047034084))",0.026841941,2020-11-10
"Map(vectorType -> dense, length -> 10, values -> List(15290.90918, 15916.26074, 15290.00684, 15701.33984, 2.9772374934E10, 4.47E-7, 0.026841941, -0.002701033, -0.009512666, 0.043536753))",0.036621327,2020-11-11
"Map(vectorType -> dense, length -> 10, values -> List(15701.29883, 16305.00391, 15534.77148, 16276.34375, 3.4175758344E10, -2.61E-6, 0.036621327, 0.026841941, -0.002701033, -0.009512666))",0.002547553,2020-11-12
"Map(vectorType -> dense, length -> 10, values -> List(16276.44043, 16463.17773, 15992.15234, 16317.80859, 3.1599492172E10, 5.94E-6, 0.002547553, 0.036621327, 0.026841941, -0.002701033))",-0.015300457,2020-11-13
"Map(vectorType -> dense, length -> 10, values -> List(16317.80859, 16317.80859, 15749.19336, 16068.13867, 2.7481710135E10, 0.0, -0.015300457, 0.002547553, 0.036621327, 0.026841941))",-0.007004594,2020-11-14
"Map(vectorType -> dense, length -> 10, values -> List(16068.13965, 16123.11035, 15793.53418, 15955.58789, 2.3653867583E10, 6.07E-8, -0.007004594, -0.015300457, 0.002547553, 0.036621327))",0.047665021,2020-11-15
"Map(vectorType -> dense, length -> 10, values -> List(15955.57715, 16816.18164, 15880.70606, 16716.11133, 3.1526766675E10, -6.73E-7, 0.047665021, -0.007004594, -0.015300457, 0.002547553))",0.055592769,2020-11-16
"Map(vectorType -> dense, length -> 10, values -> List(16685.69141, 17782.91992, 16564.54492, 17645.40625, 3.900684917E10, -0.001823114, 0.055592769, 0.047665021, -0.007004594, -0.015300457))",0.008988153,2020-11-17
"Map(vectorType -> dense, length -> 10, values -> List(17645.19141, 18393.94922, 17352.90625, 17804.00586, 4.9064800278E10, -1.22E-5, 0.008988153, 0.055592769, 0.047665021, -0.007004594))",7.3489E-4,2020-11-18
"Map(vectorType -> dense, length -> 10, values -> List(17803.86133, 18119.54688, 17382.55469, 17817.08984, 3.6985055355E10, -8.12E-6, 7.3489E-4, 0.008988153, 0.055592769, 0.047665021))",0.045137821,2020-11-19


summary,ReturnY
count,316
mean,0.0030648423259493672
stddev,0.03849974767594246
min,-0.371695386
max,0.181877557


summary,ReturnY
count,30
mean,0.006498241933333333
stddev,0.03387664317332147
min,-0.084427067
max,0.079678328


### Ethereum

In [0]:
#Final Train
trainEth = outputTraineth2.select("features", "ReturnY", "Date")
display(trainEth)

#Final Test
testEth = outputTesteth2.select("features", "ReturnY", "Date")
display(testEth)

# Displaying
display(trainEth.describe())
display(testEth.describe())

features,ReturnY,Date
"Map(vectorType -> dense, length -> 10, values -> List(144.953415, 145.529083, 142.434555, 142.869232, 7.048066973E9, 5.98E-5, -0.01431936, -0.004527733, 0.013898961, -0.018179365))",0.00172016,2019-12-14
"Map(vectorType -> dense, length -> 10, values -> List(142.86499, 143.925354, 139.426956, 143.11499, 7.235153411E9, -2.97E-5, 0.00172016, -0.01431936, -0.004527733, 0.013898961))",-0.066386903,2019-12-15
"Map(vectorType -> dense, length -> 10, values -> List(143.139526, 143.224854, 132.456665, 133.614029, 8.992282119E9, 1.71413E-4, -0.066386903, 0.00172016, -0.01431936, -0.004527733))",-0.082402575,2019-12-16
"Map(vectorType -> dense, length -> 10, values -> List(133.647186, 134.011536, 121.395081, 122.603889, 9.057166141E9, 2.48094E-4, -0.082402575, -0.066386903, 0.00172016, -0.01431936))",0.085546267,2019-12-17
"Map(vectorType -> dense, length -> 10, values -> List(122.656837, 133.394165, 119.78006, 133.092194, 1.1864518321E10, 4.31676E-4, 0.085546267, -0.082402575, -0.066386903, 0.00172016))",-0.028334179,2019-12-18
"Map(vectorType -> dense, length -> 10, values -> List(133.05278, 134.190643, 125.971664, 129.321136, 9.56469914E9, -2.96228E-4, -0.028334179, 0.085546267, -0.082402575, -0.066386903))",-0.001972462,2019-12-19
"Map(vectorType -> dense, length -> 10, values -> List(129.384369, 129.756302, 126.985352, 129.066055, 8.17359415E9, 4.88722E-4, -0.001972462, -0.028334179, 0.085546267, -0.082402575))",-0.007245158,2019-12-20
"Map(vectorType -> dense, length -> 10, values -> List(129.095963, 129.147659, 127.612091, 128.130951, 7.449064703E9, 2.31673E-4, -0.007245158, -0.001972462, -0.028334179, 0.085546267))",0.033103337,2019-12-21
"Map(vectorType -> dense, length -> 10, values -> List(128.098129, 133.160034, 127.92849, 132.372513, 9.121058481E9, -2.56225E-4, 0.033103337, -0.007245158, -0.001972462, -0.028334179))",-0.028392775,2019-12-22
"Map(vectorType -> dense, length -> 10, values -> List(132.306992, 134.73175, 127.416718, 128.61409, 9.622497693E9, -4.95219E-4, -0.028392775, 0.033103337, -0.007245158, -0.001972462))",0.003497276,2019-12-23


features,ReturnY,Date
"Map(vectorType -> dense, length -> 10, values -> List(444.166382, 453.758362, 439.600128, 449.679626, 1.2090381666E10, 7.49E-6, 0.012420148, -0.020706923, 0.040948144, -0.041797575))",0.029534171,2020-11-10
"Map(vectorType -> dense, length -> 10, values -> List(449.679657, 473.578857, 449.524933, 462.960541, 1.4075403511E10, 6.89E-8, 0.029534171, 0.012420148, -0.020706923, 0.040948144))",-0.004223386,2020-11-11
"Map(vectorType -> dense, length -> 10, values -> List(462.959534, 467.677826, 452.072418, 461.00528, 1.2877327234E10, -2.18E-6, -0.004223386, 0.029534171, 0.012420148, -0.020706923))",0.029546633,2020-11-12
"Map(vectorType -> dense, length -> 10, values -> List(461.005493, 475.217255, 457.298248, 474.626434, 1.3191505725E10, 4.62E-7, 0.029546633, -0.004223386, 0.029534171, 0.012420148))",-0.030501026,2020-11-13
"Map(vectorType -> dense, length -> 10, values -> List(474.626434, 475.161438, 452.986084, 460.149841, 1.0312037942E10, 0.0, -0.030501026, 0.029546633, -0.004223386, 0.029534171))",-0.027362302,2020-11-14
"Map(vectorType -> dense, length -> 10, values -> List(460.149902, 460.99408, 440.254333, 447.559082, 1.0308617165E10, 1.33E-7, -0.027362302, -0.030501026, 0.029546633, -0.004223386))",0.027663892,2020-11-15
"Map(vectorType -> dense, length -> 10, values -> List(447.55899, 463.831024, 445.501617, 459.940308, 1.1441239444E10, -2.06E-7, 0.027663892, -0.027362302, -0.030501026, 0.029546633))",0.044396563,2020-11-16
"Map(vectorType -> dense, length -> 10, values -> List(464.406647, 482.232208, 460.776611, 480.360077, 1.4593057877E10, 0.009617302, 0.044396563, 0.027663892, -0.027362302, -0.030501026))",-0.001823647,2020-11-17
"Map(vectorType -> dense, length -> 10, values -> List(480.346832, 491.999908, 465.830963, 479.48407, 1.7880199224E10, -2.76E-5, -0.001823647, 0.044396563, 0.027663892, -0.027362302))",-0.016379351,2020-11-18
"Map(vectorType -> dense, length -> 10, values -> List(479.481018, 480.121735, 465.704254, 471.630432, 1.2473929218E10, -6.37E-6, -0.016379351, -0.001823647, 0.044396563, 0.027663892))",0.080813564,2020-11-19


summary,ReturnY
count,316
mean,0.0045853581835443025
stddev,0.05037579196523408
min,-0.423472215
max,0.189404015


summary,ReturnY
count,30
mean,0.0084080907
stddev,0.04742481817481735
min,-0.090917668
max,0.090286337


### Theta

In [0]:
#Final Train 
trainTheta = outputTraintheta2.select("features", "ReturnY", "Date")
display(trainTheta)

#Final Test 
testTheta = outputTesttheta2.select("features", "ReturnY", "Date")
display(testTheta)

# Displaying
display(trainTheta.describe())
display(testTheta.describe())

features,ReturnY,Date
"Map(vectorType -> dense, length -> 10, values -> List(0.090143, 0.090722, 0.083036, 0.088685, 1768192.0, -2.22E-5, -0.016196128, 0.05091049, 0.032785504, 0.084311396))",0.077093082,2019-12-14
"Map(vectorType -> dense, length -> 10, values -> List(0.088692, 0.098924, 0.084474, 0.095522, 5096621.0, 7.89E-5, 0.077093082, -0.016196128, 0.05091049, 0.032785504))",-0.041770482,2019-12-15
"Map(vectorType -> dense, length -> 10, values -> List(0.095568, 0.100991, 0.088694, 0.091532, 5611689.0, 4.81333E-4, -0.041770482, 0.077093082, -0.016196128, 0.05091049))",0.032360267,2019-12-16
"Map(vectorType -> dense, length -> 10, values -> List(0.091532, 0.10091, 0.089191, 0.094494, 7840056.0, 0.0, 0.032360267, -0.041770482, 0.077093082, -0.016196128))",0.071464855,2019-12-17
"Map(vectorType -> dense, length -> 10, values -> List(0.094494, 0.104688, 0.091284, 0.101247, 6151333.0, 0.0, 0.071464855, 0.032360267, -0.041770482, 0.077093082))",0.063873497,2019-12-18
"Map(vectorType -> dense, length -> 10, values -> List(0.101247, 0.112162, 0.100012, 0.107714, 5951808.0, 0.0, 0.063873497, 0.071464855, 0.032360267, -0.041770482))",-0.01707299,2019-12-19
"Map(vectorType -> dense, length -> 10, values -> List(0.107714, 0.109948, 0.103282, 0.105875, 2764379.0, 0.0, -0.01707299, 0.063873497, 0.071464855, 0.032360267))",-0.05790791,2019-12-20
"Map(vectorType -> dense, length -> 10, values -> List(0.105819, 0.10645, 0.098511, 0.099744, 2763499.0, -5.29206E-4, -0.05790791, -0.01707299, 0.063873497, 0.071464855))",-0.024091675,2019-12-21
"Map(vectorType -> dense, length -> 10, values -> List(0.099887, 0.106496, 0.095886, 0.097341, 3098911.0, 0.001431618, -0.024091675, -0.05790791, -0.01707299, 0.063873497))",-0.070463628,2019-12-22
"Map(vectorType -> dense, length -> 10, values -> List(0.097331, 0.100891, 0.090466, 0.090482, 2753940.0, -1.02742E-4, -0.070463628, -0.024091675, -0.05790791, -0.01707299))",0.029972812,2019-12-23


features,ReturnY,Date
"Map(vectorType -> dense, length -> 10, values -> List(0.641028, 0.657987, 0.623313, 0.641003, 1.9591349E7, -1.56E-6, -4.06E-5, -0.035165933, 0.038349314, -0.053501138))",-0.044586375,2020-11-10
"Map(vectorType -> dense, length -> 10, values -> List(0.641003, 0.653566, 0.612423, 0.612423, 1.8730118E7, 0.0, -0.044586375, -4.06E-5, -0.035165933, 0.038349314))",-0.017145013,2020-11-11
"Map(vectorType -> dense, length -> 10, values -> List(0.612423, 0.626791, 0.590405, 0.601923, 1.6643669E7, 0.0, -0.017145013, -0.044586375, -4.06E-5, -0.035165933))",0.052284096,2020-11-12
"Map(vectorType -> dense, length -> 10, values -> List(0.601928, 0.634515, 0.595535, 0.633394, 2.2198608E7, 8.31E-6, 0.052284096, -0.017145013, -0.044586375, -4.06E-5))",-0.033803604,2020-11-13
"Map(vectorType -> dense, length -> 10, values -> List(0.633394, 0.63766, 0.595949, 0.611983, 9700736.0, 0.0, -0.033803604, 0.052284096, -0.017145013, -0.044586375))",-0.013719335,2020-11-14
"Map(vectorType -> dense, length -> 10, values -> List(0.611983, 0.62249, 0.59375, 0.603587, 8911596.0, 0.0, -0.013719335, -0.033803604, 0.052284096, -0.017145013))",0.025005509,2020-11-15
"Map(vectorType -> dense, length -> 10, values -> List(0.603587, 0.629743, 0.596472, 0.61868, 1.0990771E7, 0.0, 0.025005509, -0.013719335, -0.033803604, 0.052284096))",0.056659339,2020-11-16
"Map(vectorType -> dense, length -> 10, values -> List(0.629428, 0.654897, 0.621839, 0.653734, 1.7996693E7, 0.017075821, 0.056659339, 0.025005509, -0.013719335, -0.033803604))",-0.044674134,2020-11-17
"Map(vectorType -> dense, length -> 10, values -> List(0.653732, 0.677797, 0.610337, 0.624529, 1.5617187E7, -3.06E-6, -0.044674134, 0.056659339, 0.025005509, -0.013719335))",-0.001943865,2020-11-18
"Map(vectorType -> dense, length -> 10, values -> List(0.624534, 0.639607, 0.61423, 0.623315, 1.0601771E7, 8.01E-6, -0.001943865, -0.044674134, 0.056659339, 0.025005509))",0.027804561,2020-11-19


summary,ReturnY
count,316
mean,0.008329582161392403
stddev,0.07320447599758738
min,-0.453309413
max,0.369398964


summary,ReturnY
count,30
mean,0.0034768952333333334
stddev,0.0509510416220618
min,-0.107197389
max,0.105163087


# Step 2: ML Algorithm

## Importing Required Packages

In [0]:
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor, DecisionTreeRegressor, LinearRegression

In [0]:
dt = DecisionTreeRegressor(labelCol = "ReturnY", featuresCol = "features")
rf = RandomForestRegressor(labelCol = "ReturnY", featuresCol = "features")
gb = GBTRegressor(labelCol = "ReturnY", featuresCol = "features")

## Training Models

In [0]:
# Bitcoin
dt_modelBtc = dt.fit(trainBtc)
rf_modelBtc = rf.fit(trainBtc)
gb_modelBtc = gb.fit(trainBtc)

# Ethereum
dt_modelEth = dt.fit(trainEth)
rf_modelEth = rf.fit(trainEth)
gb_modelEth = gb.fit(trainEth)

# Theta
dt_modelTheta = dt.fit(trainTheta)
rf_modelTheta = rf.fit(trainTheta)
gb_modelTheta = gb.fit(trainTheta)

## Testing Models

In [0]:
# Bitcoin
dt_predictionsBtc = dt_modelBtc.transform(testBtc)
rf_predictionsBtc = rf_modelBtc.transform(testBtc)
gb_predictionsBtc = gb_modelBtc.transform(testBtc)

# Ethereum
dt_predictionsEth = dt_modelEth.transform(testEth)
rf_predictionsEth = rf_modelEth.transform(testEth)
gb_predictionsEth = gb_modelEth.transform(testEth)

# Theta
dt_predictionsTheta = dt_modelTheta.transform(testTheta)
rf_predictionsTheta = rf_modelTheta.transform(testTheta)
gb_predictionsTheta = gb_modelTheta.transform(testTheta)

## Evaluator

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="ReturnY", predictionCol="prediction", metricName="rmse")

# Bitcoin
print('Bitcoin:')
print(evaluator.evaluate(dt_predictionsBtc))
print(evaluator.evaluate(rf_predictionsBtc))
print(evaluator.evaluate(gb_predictionsBtc))

# Ethereum
print('Ethereum:')
print(evaluator.evaluate(dt_predictionsEth))
print(evaluator.evaluate(rf_predictionsEth))
print(evaluator.evaluate(gb_predictionsEth))

# Theta
print('Theta:')
print(evaluator.evaluate(dt_predictionsTheta))
print(evaluator.evaluate(rf_predictionsTheta))
print(evaluator.evaluate(gb_predictionsTheta))

Bitcoin:
0.035215985893660956
0.03640528574677352
0.03828398855111902
Ethereum:
0.06137721727207424
0.04927536011320538
0.0679509252807517
Theta:
0.055323597787211444
0.048447958485873376
0.05667084581327231

## Hyperparameter RF

In [0]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

paramGrid = ParamGridBuilder().addGrid(rf.maxDepth, [2,4,6]).addGrid(rf.maxBins, [20,60]).build()

crossval = CrossValidator(estimator = rf, estimatorParamMaps=paramGrid, evaluator = evaluator, numFolds = 10)

In [0]:
# Training Models
cv_modelTheta = crossval.fit(trainTheta)
cv_modelEth = crossval.fit(trainEth)
cv_modelBtc = crossval.fit(trainBtc)

# RMSE:
# Bitcoin
print('Bitcoin:')
cv_predictionsBtc = cv_modelBtc.transform(testBtc)
print(evaluator.evaluate(cv_predictionsBtc))

# Ethereum
print('Ethereum:')
cv_predictionsEth = cv_modelEth.transform(testEth)
print(evaluator.evaluate(cv_predictionsEth))

# Theta
print('Theta:')
cv_predictionsTheta = cv_modelTheta.transform(testTheta)
print(evaluator.evaluate(cv_predictionsTheta))

# You may need to install mlflow if this chunk is taking forever
# Even so this chunk will take 5 minutes or so

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
Bitcoin:
0.03839393151577049
Ethereum:
0.04895113745563109
Theta:
0.05044714477677209

## Elastic Net Hyperparameter Tuning

In [0]:
# Importing Packages
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(labelCol="ReturnY", featuresCol="features")
lrModel = lr.fit(trainTheta)
lrModel

# Training Models
paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.1, 0.01]).addGrid(lr.elasticNetParam, [0, 0.5, 1]).build()
crossval = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=10)
cv_modelTheta = crossval.fit(trainTheta)
cv_modelEth = crossval.fit(trainEth)
cv_modelBtc = crossval.fit(trainBtc)

# RMSE:
# Bitcoin
print('Bitcoin:')
cv2_predictionsBtc = cv_modelBtc.transform(testBtc)
print(evaluator.evaluate(cv2_predictionsBtc))

# Ethereum
print('Ethereum:')
cv2_predictionsEth = cv_modelEth.transform(testEth)
print(evaluator.evaluate(cv2_predictionsEth))

# Theta
print('Theta:')
cv2_predictionsTheta = cv_modelTheta.transform(testTheta)
print(evaluator.evaluate(cv2_predictionsTheta))

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
Bitcoin:
0.0345924050724973
Ethereum:
0.048332895468184395
Theta:
0.04896190875934517

In [0]:
# import numpy as np
# print(cv_model.getEstimatorParamMaps()[np.argmax(cv_model.avgMetrics)])
# print(cv_model.bestModel._java_obj.getRegParam())
# print(cv_model.bestModel._java_obj.getElasticNetParam())

## Basic Linear Regression (for Ethereum since this Model had the Best RMSE)

In [0]:
from pyspark.ml.regression import LinearRegression
ethlr = LinearRegression(labelCol = "ReturnY", featuresCol = "features", solver = 'l-bfgs', regParam = 0.2, elasticNetParam = 0.6)

lrModelEth = ethlr.fit(trainEth)
# printing coeff and intercept
print("Coefficients: %s" % str(lrModelEth.coefficients))
print("Intercept: %s" % str(lrModelEth.intercept))

# more details about model
trainingSummary = lrModelEth.summary
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: (10,[],[])
Intercept: 0.004585358183544301
+--------------------+
 residuals|
+--------------------+
-0.00286519818354...|
 -0.0709722611835443|
-0.08698793318354431|
 0.08096090881645569|
-0.03291953718354...|
 -0.0065578201835443|
 -0.0118305161835443|
0.028517978816455694|
 -0.0329781331835443|
-0.00108808218354...|
 -0.0248021131835443|
-0.00532408118354...|
0.002168184816455699|
0.004125127816455699|
 0.0455637708164557|
 -0.0203506281835443|
-0.02737469418354...|
0.004604820816455699|
 -0.0305163301835443|
 0.0484836198164557|
+--------------------+
only showing top 20 rows

RMSE: 0.050296
r2: 0.000000

In [0]:
#RMSE
testResultsEth = lrModelEth.evaluate(testEth)
rmseEth = testResultsEth.rootMeanSquaredError
rmseEth

Out[25]: 0.04678414493012603

# Step 3: Testing Best Models on Dataset

## Bitcoin

In [0]:
# Bitcoin Model with Lowest RMSE = ELASTIC NET HYPERPARAMETER TURNING
cv2_predictionsBtc = cv_modelBtc.transform(testBtc)
rmseBtc = evaluator.evaluate(cv2_predictionsBtc)
rmseBtc

Out[26]: 0.0345924050724973

## Ethereum

In [0]:
# Ethereum Model with Lowest RMSE = LINEAR REGRESSION
testResultsEth = lrModelEth.evaluate(testEth)
rmseEth = testResultsEth.rootMeanSquaredError
rmseEth

Out[27]: 0.04678414493012603

## Theta

In [0]:
# Theta Model with Lowest RMSE = RANDOM FOREST
rf_predictionsTheta = rf_modelTheta.transform(testTheta)
rmseTheta = evaluator.evaluate(rf_predictionsTheta)
rmseTheta

Out[28]: 0.048447958485873376

# Step 4: Average the RMSE

In [0]:
import numpy as np
rmseList = [rmseBtc, rmseEth, rmseTheta]
rmseMean = np.mean(rmseList)
print(rmseMean)

0.04327483616283224

## The Avergae RMSE for BTC, ETH, & THETA is:  *0.04327*

# Step 5: Visualizations

## Ethereum Graphs

In [0]:
print(btc2.filter(btc2["Date"] > "2020-11-09").count())
print(eth2.filter(eth2["Date"] > "2020-11-09").count())
print(theta2.filter(theta2["Date"] > "2020-11-09").count())

30
30
30

In [0]:
display(eth2.select("ReturnY", "Date"))

ReturnY,Date
0.00172016,2019-12-14
-0.066386903,2019-12-15
-0.082402575,2019-12-16
0.085546267,2019-12-17
-0.028334179,2019-12-18
-0.001972462,2019-12-19
-0.007245158,2019-12-20
0.033103337,2019-12-21
-0.028392775,2019-12-22
0.003497276,2019-12-23


In [0]:
display(eth2.select("Volume", "Date"))

Volume,Date
7048066973,2019-12-14
7235153411,2019-12-15
8992282119,2019-12-16
9057166141,2019-12-17
11864518321,2019-12-18
9564699140,2019-12-19
8173594150,2019-12-20
7449064703,2019-12-21
9121058481,2019-12-22
9622497693,2019-12-23


In [0]:
display(eth2.select("Close","Low", "High", "Date"))

Close,Low,High,Date
142.869232,142.434555,145.529083,2019-12-14
143.11499,139.426956,143.925354,2019-12-15
133.614029,132.456665,143.224854,2019-12-16
122.603889,121.395081,134.011536,2019-12-17
133.092194,119.78006,133.394165,2019-12-18
129.321136,125.971664,134.190643,2019-12-19
129.066055,126.985352,129.756302,2019-12-20
128.130951,127.612091,129.147659,2019-12-21
132.372513,127.92849,133.160034,2019-12-22
128.61409,127.416718,134.73175,2019-12-23


## Candlestick Graphs (Extra)

In [0]:
import plotly.graph_objects as go

### Bitcoin

In [0]:
#Converting dataframe from Pyspark to Pandas
df_btc = btc.toPandas()

fig = go.Figure(data=[go.Candlestick(x=df_btc['Date'],
              open=df_btc['Open'],
              high=df_btc['High'],
              low=df_btc['Low'],
              close=df_btc['Close'],
              increasing_line_color= 'magenta', decreasing_line_color= 'cyan')])

fig.update_layout(title='Bitcoin', yaxis_title='Price')

fig.show()

### Ethereum

In [0]:
#Converting dataframe from Pyspark to Pandas
df_eth = eth.toPandas()

fig = go.Figure(data=[go.Candlestick(x=df_eth['Date'],
              open=df_eth['Open'],
              high=df_eth['High'],
              low=df_eth['Low'],
              close=df_eth['Close'],
              increasing_line_color= 'magenta', decreasing_line_color= 'cyan')])

fig.update_layout(title='Etheruem', yaxis_title='Price')

fig.show()

### Theta

In [0]:
#Converting dataframe from Pyspark to Pandas
df_theta = theta.toPandas()

fig = go.Figure(data=[go.Candlestick(x=df_theta['Date'],
              open=df_theta['Open'],
              high=df_theta['High'],
              low=df_theta['Low'],
              close=df_theta['Close'],
              increasing_line_color= 'magenta', decreasing_line_color= 'cyan')])

fig.update_layout(title='Theta', yaxis_title='Price')

fig.show()

# Step 6: Options
### We chose options 1, 3, & 6 (poster is in the file)

## Option 1
##### Download (from finance.yahoo.com) and test your data on the next month of data for all three currencies.

In [0]:
# Load in new data
btcNM = spark.read.csv('/FileStore/tables/BTC_USDNM.csv',inferSchema=True,header=True)
ethNM = spark.read.csv('/FileStore/tables/ETH_USDNM.csv',inferSchema=True,header=True)
thetaNM = spark.read.csv('/FileStore/tables/THETA_USDNM.csv',inferSchema=True,header=True)

### Pretty much Step 1 again

In [0]:
display(btcNM)
btcNM.printSchema()
display(ethNM)
ethNM.printSchema()
display(thetaNM)
thetaNM.printSchema()

# Dropping NAs
btcNM = btcNM.na.drop()
ethNM = ethNM.na.drop()
thetaNM = thetaNM.na.drop()

# Asssembler
assembler = VectorAssembler(inputCols = ['Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'OCDiff',
 'Lag1',
 'Lag2',
 'Lag3',
 'Lag4'], outputCol = "features")

testbtcNM = assembler.transform(btcNM)
testethNM = assembler.transform(ethNM)
testthetaNM = assembler.transform(thetaNM)

# Final Test Data Bitcoin
testbtcNM2 = testbtcNM.select("features", "ReturnY", "Date")
display(testbtcNM2)

# Final Test Data Ethereum
testethNM2 = testethNM.select("features", "ReturnY", "Date")
display(testethNM2)

# Final Test Data Theta
testthetaNM2 = testthetaNM.select("features", "ReturnY", "Date")
display(testthetaNM)

Date,Open,High,Low,Close,Volume,ReturnY,OCDiff,Lag1,Lag2,Lag3,Lag4
12/11/2020,18263.92969,18268.45313,17619.5332,18058.9043,18058.9043,0.041240152,-5.82E-5,-0.011283218,-0.015572121,0.012705073,-0.045357601
12/12/2020,18051.32031,18919.55078,18046.04102,18803.65625,18803.65625,0.018013867,-4.20135E-4,0.041240152,-0.011283218,-0.015572121,0.012705073
12/13/2020,18806.76563,19381.53516,18734.33203,19142.38281,19142.38281,0.005446643,1.65333E-4,0.018013867,0.041240152,-0.011283218,-0.015572121
12/14/2020,19144.49219,19305.09961,19012.70898,19246.64453,19246.64453,0.008855135,1.10182E-4,0.005446643,0.018013867,0.041240152,-0.011283218
12/15/2020,19246.91992,19525.00781,19079.8418,19417.07617,19417.07617,0.097518363,1.43E-5,0.008855135,0.005446643,0.018013867,0.041240152
12/16/2020,19418.81836,21458.9082,19298.31641,21310.59766,21310.59766,0.070132451,8.97E-5,0.097518363,0.008855135,0.005446643,0.018013867
12/17/2020,21308.35156,23642.66016,21234.67578,22805.16211,22805.16211,0.014593136,-1.05409E-4,0.070132451,0.097518363,0.008855135,0.005446643
12/18/2020,22806.79688,23238.60156,22399.8125,23137.96094,23137.96094,0.031630752,7.17E-5,0.014593136,0.070132451,0.097518363,0.008855135
12/19/2020,23132.86523,24085.85547,22826.47266,23869.83203,23869.83203,-0.016444905,-2.2028E-4,0.031630752,0.014593136,0.070132451,0.097518363
12/20/2020,23861.76563,24209.66016,23147.71094,23477.29492,23477.29492,-0.028717656,-3.38047E-4,-0.016444905,0.031630752,0.014593136,0.070132451


root
-- Date: string (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: double (nullable = true)
-- ReturnY: double (nullable = true)
-- OCDiff: double (nullable = true)
-- Lag1: double (nullable = true)
-- Lag2: double (nullable = true)
-- Lag3: double (nullable = true)
-- Lag4: double (nullable = true)

Date,Open,High,Low,Close,Volume,ReturnY,OCDiff,Lag1,Lag2,Lag3,Lag4
12/11/2020,559.679199,560.376709,537.811646,545.797363,11098819124,0.041718705,1.1989E-6,-0.024802032,-0.024064691,0.033616499,-0.06254294
12/12/2020,545.578552,573.339417,545.245605,568.567322,8534557897,0.037103585,-4.01062E-4,0.041718705,-0.024802032,-0.024064691,0.033616499
12/13/2020,568.609863,593.78125,564.565979,589.663208,9070377862,-0.006193432,7.48158E-5,0.037103585,0.041718705,-0.024802032,-0.024064691
12/14/2020,589.782471,590.492981,577.118408,586.011169,8125837102,0.005707096,2.02215E-4,-0.006193432,0.037103585,0.041718705,-0.024802032
12/15/2020,586.02179,596.247742,580.628784,589.355591,9326645840,0.079453277,1.81239E-5,0.005707096,-0.006193432,0.037103585,0.041718705
12/16/2020,589.378662,636.64032,582.039124,636.181824,15817248373,0.010511357,3.91446E-5,0.079453277,0.005707096,-0.006193432,0.037103585
12/17/2020,636.154175,673.834229,628.74939,642.868958,25479532147,0.018577648,-4.34627E-5,0.010511357,0.079453277,0.005707096,-0.006193432
12/18/2020,642.916992,662.699097,632.356079,654.811951,15756303983,0.006850764,7.47126E-5,0.018577648,0.010511357,0.079453277,0.005707096
12/19/2020,654.624207,668.769592,646.616211,659.297913,12830893778,-0.031862804,-2.86797E-4,0.006850764,0.018577648,0.010511357,0.079453277
12/20/2020,659.185059,659.923706,625.014465,638.290833,13375855442,-0.044608132,-1.71202E-4,-0.031862804,0.006850764,0.018577648,0.010511357


root
-- Date: string (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: long (nullable = true)
-- ReturnY: double (nullable = true)
-- OCDiff: double (nullable = true)
-- Lag1: double (nullable = true)
-- Lag2: double (nullable = true)
-- Lag3: double (nullable = true)
-- Lag4: double (nullable = true)

Date,Open,High,Low,Close,Volume,ReturnY,OCDiff,Lag1,Lag2,Lag3,Lag4
12/11/2020,0.681331,0.690951,0.638128,0.681598,23804857,0.015604506,-0.005732896,-0.005310571,-0.054768595,-0.029138917,0.093971501
12/12/2020,0.680686,0.716078,0.680255,0.692234,14859558,0.051263012,-0.001339825,0.015604506,-0.005310571,-0.054768595,-0.029138917
12/13/2020,0.692528,0.740943,0.683198,0.72772,20230965,0.031257901,4.24532E-4,0.051263012,0.015604506,-0.005310571,-0.054768595
12/14/2020,0.72813,0.766011,0.717345,0.750467,38059917,0.016097976,5.63086E-4,0.031257901,0.051263012,0.015604506,-0.005310571
12/15/2020,0.750378,0.773199,0.730398,0.762548,24601690,0.115233402,-1.18607E-4,0.016097976,0.031257901,0.051263012,0.015604506
12/16/2020,0.762438,0.853716,0.742709,0.850419,60731400,-0.043594981,-1.44274E-4,0.115233402,0.016097976,0.031257901,0.051263012
12/17/2020,0.853707,0.905336,0.810101,0.813345,68943084,0.116208989,0.003851438,-0.043594981,0.115233402,0.016097976,0.031257901
12/18/2020,0.813159,0.921015,0.795731,0.907863,55202232,-0.009719528,-2.28738E-4,0.116208989,-0.043594981,0.115233402,0.016097976
12/19/2020,0.907115,0.94474,0.894991,0.899039,47638657,0.013752462,-8.24592E-4,-0.009719528,0.116208989,-0.043594981,0.115233402
12/20/2020,0.898679,0.948115,0.869525,0.911403,36908815,0.018407883,-4.00588E-4,0.013752462,-0.009719528,0.116208989,-0.043594981


root
-- Date: string (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: integer (nullable = true)
-- ReturnY: double (nullable = true)
-- OCDiff: double (nullable = true)
-- Lag1: double (nullable = true)
-- Lag2: double (nullable = true)
-- Lag3: double (nullable = true)
-- Lag4: double (nullable = true)

features,ReturnY,Date
"Map(vectorType -> dense, length -> 10, values -> List(18263.92969, 18268.45313, 17619.5332, 18058.9043, 18058.9043, -5.82E-5, -0.011283218, -0.015572121, 0.012705073, -0.045357601))",0.041240152,12/11/2020
"Map(vectorType -> dense, length -> 10, values -> List(18051.32031, 18919.55078, 18046.04102, 18803.65625, 18803.65625, -4.20135E-4, 0.041240152, -0.011283218, -0.015572121, 0.012705073))",0.018013867,12/12/2020
"Map(vectorType -> dense, length -> 10, values -> List(18806.76563, 19381.53516, 18734.33203, 19142.38281, 19142.38281, 1.65333E-4, 0.018013867, 0.041240152, -0.011283218, -0.015572121))",0.005446643,12/13/2020
"Map(vectorType -> dense, length -> 10, values -> List(19144.49219, 19305.09961, 19012.70898, 19246.64453, 19246.64453, 1.10182E-4, 0.005446643, 0.018013867, 0.041240152, -0.011283218))",0.008855135,12/14/2020
"Map(vectorType -> dense, length -> 10, values -> List(19246.91992, 19525.00781, 19079.8418, 19417.07617, 19417.07617, 1.43E-5, 0.008855135, 0.005446643, 0.018013867, 0.041240152))",0.097518363,12/15/2020
"Map(vectorType -> dense, length -> 10, values -> List(19418.81836, 21458.9082, 19298.31641, 21310.59766, 21310.59766, 8.97E-5, 0.097518363, 0.008855135, 0.005446643, 0.018013867))",0.070132451,12/16/2020
"Map(vectorType -> dense, length -> 10, values -> List(21308.35156, 23642.66016, 21234.67578, 22805.16211, 22805.16211, -1.05409E-4, 0.070132451, 0.097518363, 0.008855135, 0.005446643))",0.014593136,12/17/2020
"Map(vectorType -> dense, length -> 10, values -> List(22806.79688, 23238.60156, 22399.8125, 23137.96094, 23137.96094, 7.17E-5, 0.014593136, 0.070132451, 0.097518363, 0.008855135))",0.031630752,12/18/2020
"Map(vectorType -> dense, length -> 10, values -> List(23132.86523, 24085.85547, 22826.47266, 23869.83203, 23869.83203, -2.2028E-4, 0.031630752, 0.014593136, 0.070132451, 0.097518363))",-0.016444905,12/19/2020
"Map(vectorType -> dense, length -> 10, values -> List(23861.76563, 24209.66016, 23147.71094, 23477.29492, 23477.29492, -3.38047E-4, -0.016444905, 0.031630752, 0.014593136, 0.070132451))",-0.028717656,12/20/2020


features,ReturnY,Date
"Map(vectorType -> dense, length -> 10, values -> List(559.679199, 560.376709, 537.811646, 545.797363, 1.1098819124E10, 1.1989E-6, -0.024802032, -0.024064691, 0.033616499, -0.06254294))",0.041718705,12/11/2020
"Map(vectorType -> dense, length -> 10, values -> List(545.578552, 573.339417, 545.245605, 568.567322, 8.534557897E9, -4.01062E-4, 0.041718705, -0.024802032, -0.024064691, 0.033616499))",0.037103585,12/12/2020
"Map(vectorType -> dense, length -> 10, values -> List(568.609863, 593.78125, 564.565979, 589.663208, 9.070377862E9, 7.48158E-5, 0.037103585, 0.041718705, -0.024802032, -0.024064691))",-0.006193432,12/13/2020
"Map(vectorType -> dense, length -> 10, values -> List(589.782471, 590.492981, 577.118408, 586.011169, 8.125837102E9, 2.02215E-4, -0.006193432, 0.037103585, 0.041718705, -0.024802032))",0.005707096,12/14/2020
"Map(vectorType -> dense, length -> 10, values -> List(586.02179, 596.247742, 580.628784, 589.355591, 9.32664584E9, 1.81239E-5, 0.005707096, -0.006193432, 0.037103585, 0.041718705))",0.079453277,12/15/2020
"Map(vectorType -> dense, length -> 10, values -> List(589.378662, 636.64032, 582.039124, 636.181824, 1.5817248373E10, 3.91446E-5, 0.079453277, 0.005707096, -0.006193432, 0.037103585))",0.010511357,12/16/2020
"Map(vectorType -> dense, length -> 10, values -> List(636.154175, 673.834229, 628.74939, 642.868958, 2.5479532147E10, -4.34627E-5, 0.010511357, 0.079453277, 0.005707096, -0.006193432))",0.018577648,12/17/2020
"Map(vectorType -> dense, length -> 10, values -> List(642.916992, 662.699097, 632.356079, 654.811951, 1.5756303983E10, 7.47126E-5, 0.018577648, 0.010511357, 0.079453277, 0.005707096))",0.006850764,12/18/2020
"Map(vectorType -> dense, length -> 10, values -> List(654.624207, 668.769592, 646.616211, 659.297913, 1.2830893778E10, -2.86797E-4, 0.006850764, 0.018577648, 0.010511357, 0.079453277))",-0.031862804,12/19/2020
"Map(vectorType -> dense, length -> 10, values -> List(659.185059, 659.923706, 625.014465, 638.290833, 1.3375855442E10, -1.71202E-4, -0.031862804, 0.006850764, 0.018577648, 0.010511357))",-0.044608132,12/20/2020


Date,Open,High,Low,Close,Volume,ReturnY,OCDiff,Lag1,Lag2,Lag3,Lag4,features
12/11/2020,0.681331,0.690951,0.638128,0.681598,23804857,0.015604506,-0.005732896,-0.005310571,-0.054768595,-0.029138917,0.093971501,"Map(vectorType -> dense, length -> 10, values -> List(0.681331, 0.690951, 0.638128, 0.681598, 2.3804857E7, -0.005732896, -0.005310571, -0.054768595, -0.029138917, 0.093971501))"
12/12/2020,0.680686,0.716078,0.680255,0.692234,14859558,0.051263012,-0.001339825,0.015604506,-0.005310571,-0.054768595,-0.029138917,"Map(vectorType -> dense, length -> 10, values -> List(0.680686, 0.716078, 0.680255, 0.692234, 1.4859558E7, -0.001339825, 0.015604506, -0.005310571, -0.054768595, -0.029138917))"
12/13/2020,0.692528,0.740943,0.683198,0.72772,20230965,0.031257901,4.24532E-4,0.051263012,0.015604506,-0.005310571,-0.054768595,"Map(vectorType -> dense, length -> 10, values -> List(0.692528, 0.740943, 0.683198, 0.72772, 2.0230965E7, 4.24532E-4, 0.051263012, 0.015604506, -0.005310571, -0.054768595))"
12/14/2020,0.72813,0.766011,0.717345,0.750467,38059917,0.016097976,5.63086E-4,0.031257901,0.051263012,0.015604506,-0.005310571,"Map(vectorType -> dense, length -> 10, values -> List(0.72813, 0.766011, 0.717345, 0.750467, 3.8059917E7, 5.63086E-4, 0.031257901, 0.051263012, 0.015604506, -0.005310571))"
12/15/2020,0.750378,0.773199,0.730398,0.762548,24601690,0.115233402,-1.18607E-4,0.016097976,0.031257901,0.051263012,0.015604506,"Map(vectorType -> dense, length -> 10, values -> List(0.750378, 0.773199, 0.730398, 0.762548, 2.460169E7, -1.18607E-4, 0.016097976, 0.031257901, 0.051263012, 0.015604506))"
12/16/2020,0.762438,0.853716,0.742709,0.850419,60731400,-0.043594981,-1.44274E-4,0.115233402,0.016097976,0.031257901,0.051263012,"Map(vectorType -> dense, length -> 10, values -> List(0.762438, 0.853716, 0.742709, 0.850419, 6.07314E7, -1.44274E-4, 0.115233402, 0.016097976, 0.031257901, 0.051263012))"
12/17/2020,0.853707,0.905336,0.810101,0.813345,68943084,0.116208989,0.003851438,-0.043594981,0.115233402,0.016097976,0.031257901,"Map(vectorType -> dense, length -> 10, values -> List(0.853707, 0.905336, 0.810101, 0.813345, 6.8943084E7, 0.003851438, -0.043594981, 0.115233402, 0.016097976, 0.031257901))"
12/18/2020,0.813159,0.921015,0.795731,0.907863,55202232,-0.009719528,-2.28738E-4,0.116208989,-0.043594981,0.115233402,0.016097976,"Map(vectorType -> dense, length -> 10, values -> List(0.813159, 0.921015, 0.795731, 0.907863, 5.5202232E7, -2.28738E-4, 0.116208989, -0.043594981, 0.115233402, 0.016097976))"
12/19/2020,0.907115,0.94474,0.894991,0.899039,47638657,0.013752462,-8.24592E-4,-0.009719528,0.116208989,-0.043594981,0.115233402,"Map(vectorType -> dense, length -> 10, values -> List(0.907115, 0.94474, 0.894991, 0.899039, 4.7638657E7, -8.24592E-4, -0.009719528, 0.116208989, -0.043594981, 0.115233402))"
12/20/2020,0.898679,0.948115,0.869525,0.911403,36908815,0.018407883,-4.00588E-4,0.013752462,-0.009719528,0.116208989,-0.043594981,"Map(vectorType -> dense, length -> 10, values -> List(0.898679, 0.948115, 0.869525, 0.911403, 3.6908815E7, -4.00588E-4, 0.013752462, -0.009719528, 0.116208989, -0.043594981))"


### Results

#### Bitcoin

In [0]:
#RMSE
test_resultsbtcNM = cv_modelBtc.transform(testbtcNM2)
rmse_btcNM = evaluator.evaluate(test_resultsbtcNM)
rmse_btcNM

Out[52]: 0.05066156491797875

#### Etereum

In [0]:
#RMSE
test_resultsethNM = lrModelEth.evaluate(testethNM2)
rmse_ethNM = test_resultsethNM.rootMeanSquaredError
print(rmse_ethNM)

# Displaying residuals
display(test_resultsethNM.residuals)

0.0669245273541424

residuals
0.0371333468164557
0.0325182268164557
-0.010778790183544301
0.0011217378164556993
0.0748679188164557
0.0059259988164557
0.013992289816455698
0.002265405816455699
-0.036448162183544304
-0.049193490183544304


#### Theta

In [0]:
# Finding rmse
test_resultsthetaNM = rf_modelTheta.transform(testthetaNM2)
rmse_thetaNM = evaluator.evaluate(test_resultsthetaNM)
rmse_thetaNM

Out[54]: 0.10895255591091357

## Option 3

##### Find a single algorithm that works well on all of the 3 crypto-currencies and see how it works on other crypto-currencies (download another crypto currency and predict it for 30 days). You may choose your timeframes for both train and test.

### Loading in and Transforming Dogecoin Dataset

In [0]:
#Loading in dataset
doge = spark.read.csv('/FileStore/tables/DOGE_USD.csv',inferSchema=True,header=True)
#display(doge)
doge.printSchema()

root
-- Date: string (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: long (nullable = true)
-- ReturnY: double (nullable = true)
-- OCDIff: double (nullable = true)
-- Lag1: double (nullable = true)
-- Lag2: double (nullable = true)
-- Lag3: double (nullable = true)
-- Lag4: double (nullable = true)

In [0]:
# Dropping na
doge2=doge.na.drop()

# Changing the date datatype to date
from pyspark.sql.functions import to_date
doge2 = doge2.withColumn("Date", to_date(doge2['Date'], "M/d/yy"))
doge2.printSchema()

# Splitting into traingin and testing dataset
train_datadoge2 = doge2.limit(315)
test_datadoge2 = doge2.orderBy(doge2["Date"].desc()).limit(30).orderBy("Date")

root
-- Date: date (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: long (nullable = true)
-- ReturnY: double (nullable = true)
-- OCDIff: double (nullable = true)
-- Lag1: double (nullable = true)
-- Lag2: double (nullable = true)
-- Lag3: double (nullable = true)
-- Lag4: double (nullable = true)

In [0]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ['Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'OCDIff',
 'Lag1',
 'Lag2',
 'Lag3',
 'Lag4'], outputCol = "features")

outputTraindoge2 = assembler.transform(train_datadoge2)
outputTestdoge2 = assembler.transform(test_datadoge2)

#Final Train DOGE
trainDoge = outputTraindoge2.select("features", "ReturnY", "Date")
#Final Test DOGE
testDoge = outputTestdoge2.select("features", "ReturnY", "Date")

# Example
display(trainDoge)

features,ReturnY,Date
"Map(vectorType -> dense, length -> 10, values -> List(0.001986, 0.002044, 0.00196, 0.002012, 1.16283369E8, 0.005538771, 0.007511267, 0.008585859, 0.061662198, -0.007978723))",0.004473161,2020-04-08
"Map(vectorType -> dense, length -> 10, values -> List(0.002013, 0.002033, 0.001985, 0.002021, 1.09954538E8, -4.96771E-4, 0.004473161, 0.007511267, 0.008585859, 0.061662198))",-0.026224641,2020-04-09
"Map(vectorType -> dense, length -> 10, values -> List(0.002021, 0.002023, 0.001891, 0.001968, 1.2210879E8, 0.0, -0.026224641, 0.004473161, 0.007511267, 0.008585859))",0.011178862,2020-04-10
"Map(vectorType -> dense, length -> 10, values -> List(0.001968, 0.002004, 0.001941, 0.00199, 1.61367396E8, 0.0, 0.011178862, -0.026224641, 0.004473161, 0.007511267))",0.0,2020-04-11
"Map(vectorType -> dense, length -> 10, values -> List(0.001989, 0.002009, 0.001951, 0.00199, 1.69892709E8, 5.02765E-4, 0.0, 0.011178862, -0.026224641, 0.004473161))",-0.015577889,2020-04-12
"Map(vectorType -> dense, length -> 10, values -> List(0.00199, 0.00199, 0.001894, 0.001959, 1.6735765E8, 0.0, -0.015577889, 0.0, 0.011178862, -0.026224641))",0.002552323,2020-04-13
"Map(vectorType -> dense, length -> 10, values -> List(0.001958, 0.00198, 0.001935, 0.001964, 1.88937615E8, 5.10725E-4, 0.002552323, -0.015577889, 0.0, 0.011178862))",-0.012219959,2020-04-14
"Map(vectorType -> dense, length -> 10, values -> List(0.001963, 0.001975, 0.001906, 0.00194, 1.69903672E8, 5.09424E-4, -0.012219959, 0.002552323, -0.015577889, 0.0))",0.045360825,2020-04-15
"Map(vectorType -> dense, length -> 10, values -> List(0.001971, 0.002033, 0.001964, 0.002011, 1.63753943E8, 0.0, 0.020294267, -0.00554995, -0.036929057, -0.001455604))",0.017901542,2020-04-22
"Map(vectorType -> dense, length -> 10, values -> List(0.002011, 0.002123, 0.002005, 0.002047, 1.7900035E8, 0.0, 0.017901542, 0.020294267, -0.00554995, -0.036929057))",0.027357108,2020-04-23


In [0]:
from pyspark.ml.regression import LinearRegression

dogeLR = LinearRegression(labelCol = "ReturnY", featuresCol = "features", solver = 'l-bfgs', regParam=0.7)

lrModelDoge = dogeLR.fit(trainDoge)
# printing coeff and intercept
print("Coefficients: %s" % str(lrModelDoge.coefficients))
print("Intercept: %s" % str(lrModelDoge.intercept))

# more details about model
trainingSummary = lrModelDoge.summary
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: [-0.00872904356885888,-0.002856347070925378,-0.023427173666502356,0.006829590126776283,-6.325500156720279e-13,0.1331572256838448,0.018137690620592683,0.002616455060123739,0.005381066165874766,-0.007512860778298973]
Intercept: 0.019318547828731315
+--------------------+
 residuals|
+--------------------+
-0.01600446173661...|
-0.04503513684883108|
-0.00752012963351...|
-0.01926347316072...|
-0.03465522821098...|
-0.01658094525903293|
-0.03135388492551737|
 0.02643488489001497|
-0.00142392270021...|
0.007583026405368093|
-0.00127758509843...|
 0.1051202702880282|
-0.04387743945095175|
-0.02563065645199...|
 0.05660322632918762|
-0.04877190566982...|
0.002017571928318942|
0.002274333824202758|
-0.03405011496248288|
-0.03079969606797...|
+--------------------+
only showing top 20 rows

RMSE: 0.181384
r2: 0.010896

In [0]:
#RMSE
testResultsDoge = lrModelDoge.evaluate(testDoge)
rmseDoge = testResultsDoge.rootMeanSquaredError
rmseDoge

Out[69]: 0.058801133560562255

### RMSE = .05880
##### If we were to guess the reason why this model's RMSE is higher than the other cryptocurrencies is because of Dogecoin's extreme volatility.

### Doge Candlestock Graph

In [0]:
#Converting dataframe from Pyspark to Pandas
df_doge = doge.toPandas()

fig = go.Figure(data=[go.Candlestick(x=df_doge['Date'],
              open=df_doge['Open'],
              high=df_doge['High'],
              low=df_doge['Low'],
              close=df_doge['Close'],
              increasing_line_color= 'magenta', decreasing_line_color= 'cyan')])

fig.update_layout(title='Dogecoin to the Moon', yaxis_title='Price')

fig.show()

### Predicting Next 30 Days

In [0]:
lr_predictions = lrModelDoge.transform(testDoge)
lr_predictions.select("prediction","ReturnY","features").show(30)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="ReturnY",metricName="r2")
print("R^2 = %g" % lr_evaluator.evaluate(lr_predictions))

+--------------------+------------+--------------------+
 prediction| ReturnY| features|
+--------------------+------------+--------------------+
0.017156772645292504| 0.027716745|[0.050028,0.05085...|
0.017384787570601724| 0.022320728|[0.049601,0.05239...|
0.017819175083312714| 0.188538429|[0.05098,0.052141...|
0.019341536811785555|-0.063778269|[0.052123,0.06194...|
 0.01503765038198189|-0.034380496|[0.062104,0.06226...|
 0.01691020948937858|-0.001535605|[0.057964,0.05861...|
0.015179627271763176|-0.010175614|[0.055977,0.05669...|
0.017052440941745736| 0.127969792|[0.055921,0.05698...|
 0.01915274320746398| -0.06150692|[0.055353,0.06243...|
0.015159019445515374|-0.025669033|[0.062384,0.06305...|
0.016912732960521242| 0.026608044|[0.058531,0.05968...|
0.015998245081072497|-0.016277919|[0.057086,0.05892...|
0.017255898986918258|-0.004683191|[0.05861,0.058866...|
0.017433904559166824| 0.016468292|[0.05764,0.058828...|
 0.01712202380007528| 0.010852421|[0.057377,0.05972...|
0.017403371946624505|-0.029290548|[0.058315,0.06063...|
0.016814181734545592|-0.039696684|[0.05897,0.059521...|
0.016381890157451607|-0.025581311|[0.057259,0.05773...|
0.016389861892348926| -0.03461797|[0.054941,0.05608...|
0.016632715640287638|-0.004912769|[0.053528,0.05649...|
 0.0172074826766403| 0.048670502|[0.051699,0.05240...|
0.018304816011373216| 0.007877372|[0.051433,0.05402...|
0.017863369347821394|-0.012486897|[0.05394,0.055435...|
0.017604116465622065| 0.006592424|[0.054381,0.05475...|
0.017290287503200087|-0.002812107|[0.053678,0.05465...|
 0.0171773324120682|-0.004508349|[0.054052,0.05543...|
 0.01745631205092968| 0.155226718|[0.053902,0.05446...|
 0.0168815657691438|-0.069725422|[0.053655,0.07011...|
0.015456835954448044|-0.032255827|[0.061968,0.06224...|
0.017115496958651735| 0.019371371|[0.057658,0.05948...|
+--------------------+------------+--------------------+

R^2 = -0.0312873